In [ ]:
from gerrychain.random import random

SEED = 0
while (SEED % 2 == 0):
    SEED = random.randrange(1000000001,9999999999)
#SEED = 7473378879
random.seed(SEED)
print(SEED)

import matplotlib.pyplot as plt
from gerrychain import Graph
import geopandas as gp
import time
import csv
import os
import numpy as np
import collections as col



# Holds all parameter info from read-in parameter list
class PARAMS:
    def __init__(self, parameters):
        self.PlanFolder = parameters[0][1]
        self.maps = []
        self.StateFolder = parameters[1][1]
        self.shapefile = []
        self.OutputFolder = parameters[2][1]
        self.output_name = ''
        
        if len(parameters[2]) > 2:
            self.output_name = parameters[2][2]
        
        self.num_iterations = int(parameters[3][1])
        self.objective = parameters[4][1]
        self.constraints = [c for c in parameters[5][1:] if (c != '')]
        self.max_pop_dev = float(parameters[6][1])
        self.ideal_pop = 0
        self.perim_mult = float(parameters[7][1])
        self.perim_threshold = 0
        self.cut_edges_mult = float(parameters[8][1])
        self.cut_edges_threshold = 0
        self.eg_threshold = float(parameters[9][1])
        self.seg_threshold = float(parameters[10][1])
        self.mm_threshold = float(parameters[11][1])
        self.pa_threshold = float(parameters[12][1])
        self.cmpttv_margin = float(parameters[13][1])
        self.edgeBonus = float(parameters[14][1])
        self.edgePenalty = float(parameters[15][1])
        self.converge = parameters[16][1]
        self.convergence_threshold = 1      
        self.repeated_runs = int(parameters[17][1])
        self.time_limit_bool = parameters[18][1]
        self.time_limit = 1
        self.time_start = 0
        self.search_type = parameters[19][1]
        self.temperature_initial = 1.0
        self.temperature = 1.0
        self.cooling = 0.999
        self.epsilon = 0.001
        self.forbidden = []
        
        if len(parameters[20]) >= 3:
            if (parameters[20][1] != '') and (parameters[20][2] != ''):
                for entry in parameters[20:]:
                    self.forbidden.append(entry[1:3])

    def set_maps(self,file_list):
        self.maps = []
        for f in file_list:
            self.maps.append(f)
        
    def set_shapefile(self,file_list):
        self.shapefile = []
        for f in file_list:
            self.shapefile.append(f)
    
    
# Holds all district info from plan
class PLAN_VALUES:
    def __init__(self):
        self.numDistricts = 0
        self.distCounties = []
        self.borderNodes = [] # List of all border nodes
        self.borderNodesByDist = {} # Records border nodes for each district (used to calculate initial district perimeters & auxH)
        self.borderNodesYesNo = {} # For every node, record whether it's a border node (can be adapted to record # of districts node is adjacent to, besides its own)
        self.auxH = {}
        self.distPop = []
        self.meanDistPop = 0
        self.distPopPercent = []
        self.distPerimeter = []
        self.CutEdges = 0
        self.fracDem = []
        self.numDem = []
        self.numRep = []
        self.totalVotesByDistrict = []
        self.wastedNominal = []
        self.wastedShift = []
        self.compFrac = []
        self.fracW = []
        self.fracB = []
        self.fracL = []
        self.numPlurB = 0
        self.numPlurL = 0
        self.numPlurW = 0

    
    
# Input: Lists with number of dem/rep voters in each district
# Output: Dict of wasted votes in each district (nominal)
def initialize_wasted(ND,NR):
    
    # Store wasted votes for each district
    WASTED_NOMINAL = {}

    for i in range(0,len(ND)):
        if ND[i] >= NR[i]:
            WASTED_NOMINAL[i+1] = .5*(ND[i]-3*NR[i])
        else:
            WASTED_NOMINAL[i+1] = .5*(3*ND[i]-NR[i])
                
    return WASTED_NOMINAL


# Input: Dict of wasted votes in each district, total votes
# Output: Efficiency gap value
def update_EG_fast(wasted, total):

    sumWasted = 0.0
    for i in wasted:
        sumWasted += wasted[i]

    return (round(abs(sumWasted)/total,10))
    

# Input: district From/To, fracDem list From /To, total votes From/To, # districts, wasted list, total votes
# Output: New efficiency gap value after a flip iteration
def update_EG_fast_it(From,To,FDFrom,FDTo,TVFrom,TVTo,num,waste,total):
    
    NDFrom = FDFrom*TVFrom
    NRFrom = (1-FDFrom)*TVFrom
    
    NDTo = FDTo*TVTo
    NRTo = (1-FDTo)*TVTo
    
    if NDFrom >= NRFrom:
        wastedFrom = .5*(NDFrom-3*NRFrom)
    else:
        wastedFrom = .5*(3*NDFrom-NRFrom)
        
        
    if NDTo >= NRTo:
        wastedTo = .5*(NDTo-3*NRTo)
    else:
        wastedTo = .5*(3*NDTo-NRTo)
        

    sumWasted = 0.0
    for i in range(1,num):
        if i == From:
            sumWasted += wastedFrom
        elif i == To:
            sumWasted += wastedTo
        else:
            sumWasted += waste[i]
            

    return (round(abs(sumWasted)/total,10)),wastedFrom,wastedTo

    
    
# Input: Lists with fraction of democratic voters and total number of voters in each district
# Output: Dict of wasted votes in each district for each vote-share shift
def initialize_wastedShift(FD,TV):
    
     # Shifted Efficiency Gap objective function (MO)
    percentShiftsNeg = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0]
    percentShiftsPos = [0.01,0.02,0.03,0.04,0.05]
    #allVotes = sum(TV)
    WASTED_ALL = {}

    for s in percentShiftsNeg:
        fracDem_shift = [max(fd+s,0.0) for fd in FD]
        numDem_shift = []
        numRep_shift = []
        for i in range(0,len(TV)):
            numDem_shift.append(fracDem_shift[i]*TV[i])
            numRep_shift.append((1-fracDem_shift[i])*TV[i])
        
        #wasted_shift = []
        for i in range(0,len(numDem_shift)):
            if numDem_shift[i] >= numRep_shift[i]:
                #wasted_shift.append(.5*(numDem_shift[i]-3*numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(numDem_shift[i]-3*numRep_shift[i])
            else:
                #wasted_shift.append(.5*(3*numDem_shift[i]-numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(3*numDem_shift[i]-numRep_shift[i])
        
 
    for s in percentShiftsPos:
        fracDem_shift = [min(fd+s,1.0) for fd in FD]
        numDem_shift = []
        numRep_shift = []
        for i in range(0,len(TV)):
            numDem_shift.append(fracDem_shift[i]*TV[i])
            numRep_shift.append((1-fracDem_shift[i])*TV[i])
            
        #wasted_shift = []
        for i in range(0,len(numDem_shift)):
            if numDem_shift[i] >= numRep_shift[i]:
                #wasted_shift.append(.5*(numDem_shift[i]-3*numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(numDem_shift[i]-3*numRep_shift[i])
            else:
                #wasted_shift.append(.5*(3*numDem_shift[i]-numRep_shift[i]))
                WASTED_ALL[(i+1,s)] = .5*(3*numDem_shift[i]-numRep_shift[i])
                
    return WASTED_ALL



# Input: Dict of wasted votes shift for each district, total votes, number of districts
# Output: Shifted efficiency gap value
def update_ShiftedEG_fast(wasted,total,num):
    
    percentShifts = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0,0.01,0.02,0.03,0.04,0.05]
    EG_shift = []
    
    for s in percentShifts:
        sumWasted = 0.0
        for i in range(1,num):
            sumWasted += wasted[(i,s)]
            
        EG_shift.append(round(abs(sumWasted)/total,10))
        
#     print('-----')
#     for val in EG_shift:
#         print(val)
        
    return max(EG_shift)
    #return EG_shift[5] # Quick change to get nominal EG value!
    

# Input: district From/To, fracDem list From /To, total votes From/To, # districts, wasted list, total votes
# Output: New efficiency gap value after a flip iteration
def update_ShiftedEG_fast_it(From,To,FDFrom,FDTo,TVFrom,TVTo,num,waste,total):
    
    percentShiftsNeg = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0]
    percentShiftsPos = [0.01,0.02,0.03,0.04,0.05]
    percentShifts = [-0.05,-0.04,-0.03,-0.02,-0.01,0.0,0.01,0.02,0.03,0.04,0.05]
    EG_shift = []
    
    wastedFrom = {}
    wastedTo = {}
    
    for s in percentShiftsNeg:
        numDemFrom_shift = (max(FDFrom+s,0.0))*TVFrom
        numRepFrom_shift = (1-max(FDFrom+s,0.0))*TVFrom
        
        numDemTo_shift = (max(FDTo+s,0.0))*TVTo
        numRepTo_shift = (1-max(FDTo+s,0.0))*TVTo
        
        if numDemFrom_shift >= numRepFrom_shift:
            wastedFrom[s] = .5*(numDemFrom_shift-3*numRepFrom_shift)
        else:
            wastedFrom[s] = .5*(3*numDemFrom_shift-numRepFrom_shift)
            
        if numDemTo_shift >= numRepTo_shift:
            wastedTo[s] = .5*(numDemTo_shift-3*numRepTo_shift)
        else:
            wastedTo[s] = .5*(3*numDemTo_shift-numRepTo_shift)
            
            
    for s in percentShiftsPos:
        numDemFrom_shift = (min(FDFrom+s,1.0))*TVFrom
        numRepFrom_shift = (1-min(FDFrom+s,1.0))*TVFrom
        
        numDemTo_shift = (min(FDTo+s,1.0))*TVTo
        numRepTo_shift = (1-min(FDTo+s,1.0))*TVTo
        
        if numDemFrom_shift >= numRepFrom_shift:
            wastedFrom[s] = .5*(numDemFrom_shift-3*numRepFrom_shift)
        else:
            wastedFrom[s] = .5*(3*numDemFrom_shift-numRepFrom_shift)
            
        if numDemTo_shift >= numRepTo_shift:
            wastedTo[s] = .5*(numDemTo_shift-3*numRepTo_shift)
        else:
            wastedTo[s] = .5*(3*numDemTo_shift-numRepTo_shift)
            
            
    for s in percentShifts:
        sumWasted = 0.0
        for i in range(1,num):
            if i == From:
                sumWasted += wastedFrom[s]
            elif i == To:
                sumWasted += wastedTo[s]
            else:
                sumWasted += waste[(i,s)]
            
        EG_shift.append(round(abs(sumWasted)/total,10))
        
    return max(EG_shift),wastedFrom,wastedTo
    #return EG_shift[5],wastedFrom,wastedTo # Quick change to get nominal EG value!
    
    
# Input: List with fraction of dem voters in each district
# Output: partisan asymmetry value
def update_PA(FD):
    FD_copy = FD.copy()
    curve_Dem = []
    all_won = False
    all_lost = False
    
    # Increase average vote-share until dems win all seats, recording breakpoints for seat gains
    while not all_won:
        change_increase = [(0.5 - fd) for fd in FD_copy if fd < 0.5]
        if len(change_increase) == 0:
            all_won = True
            voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
            seatShare_Dem = (len(FD_copy) - len(change_increase))/(len(FD_copy))
            curve_Dem.append([voteShare_Dem,seatShare_Dem])
            continue
            
        delta_increase = min(change_increase)
        
        voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
        seatShare_Dem = (len(FD_copy) - len(change_increase))/(len(FD_copy))
        curve_Dem.append([voteShare_Dem,seatShare_Dem])
        
        FD_copy = [(fd + delta_increase) for fd in FD_copy] # Increase district vote-shares by enough for dems to win one more seat
        
    current = curve_Dem[0]
    del(curve_Dem[0])
    FD_copy = FD.copy()
    
    # Indicator to not record point in first loop iteration, since it's not a breakpoint
    goAhead = False
    
    # Decrease average vote-share until dems lose all seats, recording breakpoints for seat losses
    while not all_lost:
        change_decrease = [(fd - 0.5) for fd in FD_copy if fd > 0.5]
        if len(change_decrease) == 0:
            all_lost = True
            voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
            seatShare_Dem = (len(change_decrease)+1)/(len(FD_copy))
            curve_Dem.append([voteShare_Dem,seatShare_Dem])
            continue
            
        delta_decrease = min(change_decrease)
        
        voteShare_Dem = (sum(FD_copy))/(len(FD_copy))
        seatShare_Dem = (len(change_decrease)+1)/(len(FD_copy))
        if goAhead:
            curve_Dem.append([voteShare_Dem,seatShare_Dem])
            
        goAhead = True
        
        FD_copy = [(fd - delta_decrease) for fd in FD_copy]
    
    
    curve_Dem.sort()
    
    # Create rep curve from dem curve
    curve_Rep = [[1-cd[0],1-cd[1]+(1/len(FD_copy))] for cd in curve_Dem]
    curve_Rep.sort()
    
    # Calculate area between curves
    integ = 0
    for i in range(0,len(curve_Dem)):
        dem = curve_Dem[i]
        rep = curve_Rep[i]
        rect = (1.0/(len(FD_copy)))*(abs(dem[0]-rep[0]))
        integ += rect
        
        
#     # Plot dem and rep curves
#     x = [cd[0] for cd in curve_Dem]
#     y = [cd[1]-1.0/(len(FD_copy)) for cd in curve_Dem]
#     x.append(1.0)
#     y.append(1.0)
#     plt.step(x,y,'b',linewidth=.5,label='Party A')
    
#     x = [cd[0] for cd in curve_Rep]
#     y = [cd[1]-1.0/(len(FD_copy)) for cd in curve_Rep]
#     x.append(1.0)
#     y.append(1.0)
#     plt.step(x,y,'r--',linewidth=.5,label='Party B')
#     plt.axis([0,1,0,1])
#     plt.legend()
#     plt.xlabel('Average Vote-Share')
#     plt.ylabel('Seat-Share')
#     plt.show()
    
    return (integ)

        
        
    
# Input: Gerrychain graph object, adjacency dict
# Output: Gerrychain graph object with merged cut-vertices, adjacency dict, cut vertices dict
def merge_cut_vertices(graph,adj):
    
    # CHECK ALL CUT-VERTEX CANDIDATES -----------------------------------------------------------------

    # Check contiguity of aug. neighborhood of candidate upon removal of candidate using a breadth-first search

    cutVertices = {}

    for node in graph.nodes:
        if node != 'dummy':
            chosen_nbhd = [val for val in graph.nodes[node]['neighbors']]
            if 'dummy' in chosen_nbhd:
                chosen_nbhd.remove('dummy')
            chosen_nbhd_aug = [val for val in graph.nodes[node]['neighbors_aug']]
            if 'dummy' in chosen_nbhd_aug:
                chosen_nbhd_aug.remove('dummy')
            chosen_nbhd_aug = set(chosen_nbhd_aug)

            isPath = True
            x = chosen_nbhd[0]
            discovered = [x]
            bfsQ = col.deque()
            bfsQ.append(x)
            found = False
            while len(bfsQ) > 0:
                v = bfsQ.popleft()
                vNbhd = [val for val in graph.nodes[v]['neighbors']]
                if 'dummy' in vNbhd:
                    vNbhd.remove('dummy')
                vNbhd = set(vNbhd)
                vBoth = list(vNbhd.intersection(chosen_nbhd_aug)) # For aug-nbhd BFS
                if node in vBoth:
                    vBoth.remove(node)

                for w in vBoth:
                    if w not in discovered:
                        discovered.append(w)
                        bfsQ.append(w)


            for y in chosen_nbhd:
                if y not in discovered:
                    isPath = False
                    break


            # Add node to dictionary of cut-vertices if aug. neighborhood will become discontiguous
            if not isPath:
                cutVertices[node] = [] # this list will contain the nodes it surrounds


    print('Number of cut-vertices: ',len(cutVertices))
    
    
    # IDENTIFY UNITS SURROUNDED BY CUT-VERTICES -------------------------------------------------------

    # Start breadth-first search at a neighbor of a cut-vertex.
    # If search continues to reach more than 75 nodes, starting node is not surrounded.
    # If search stops before reaching 75 nodes, starting node is surrounded.

    for node in cutVertices:
        
        chosen_nbhd = [val for val in graph.nodes[node]['neighbors']]
        if 'dummy' in chosen_nbhd:
            chosen_nbhd.remove('dummy')
#         print(chosen_nbhd)
            
        for x in chosen_nbhd:
            
#             print('x: ',x)

            isSurrounded = True
            discovered = [x]
            bfsQ = col.deque()
            bfsQ.append(x)
            found = False
            while len(bfsQ) > 0:
                v = bfsQ.popleft()
                vBoth = [val for val in graph.nodes[v]['neighbors']]
                if 'dummy' in vBoth:
                    vBoth.remove('dummy')
                if node in vBoth:
                    vBoth.remove(node)

                for w in vBoth:
                    if w not in discovered:
                        discovered.append(w)
                        bfsQ.append(w)
#                         print('w: ',w)


                if len(discovered) > 75: # Large number due to odd peninsula of blocks in MO
                    isSurrounded = False
                    break
                    
#             print('isSurrounded: ',isSurrounded)

            # Record all nodes that cut-vertex "node" surrounds
            if isSurrounded:
                for d in discovered:
                    if d not in cutVertices[node]:
                        cutVertices[node].append(d)


    # Keep list of all surrounded units
    surrounded = []

    for cv in cutVertices:
        for unit in cutVertices[cv]:
            if unit not in surrounded:
                surrounded.append(unit)


    # If a unit is both a cut-vertex and surrounded, remove it from cutVertices
    for unit in surrounded:
        if unit in cutVertices:
            del cutVertices[unit]

    print('Number of cut-vertices (post-cleaning): ',len(cutVertices))
    
    # Make cut vertices dict with GEOIDs for surrounded units, so able to output full final assignment
    cutVertices_ID = {}
    for cv in cutVertices:
        cutVertices_ID[cv] = [graph.nodes[u]['GEOID20'] for u in cutVertices[cv]]
    
        
#     surrounded_outFile = open('IL_SurroundedTracts.csv','w')
#     surrounded_writer = csv.writer(surrounded_outFile,delimiter=',')
    
#     surrounded_writer.writerow(['GEOID20','Cut/Surrounded'])
    
#     for cv in cutVertices:
#         surrounded_writer.writerow([cv,'0'])
#         for surr in cutVertices[cv]:
#             surrounded_writer.writerow([surr,'1'])
        
#     surrounded_outFile.close()

    
    
    # MERGE DATA OF SURROUNDED UNITS WITH SURROUNDING UNIT --------------------------------------------

    #print(len(surrounded))
    #print(len(graph.nodes))
    
    for chosen in cutVertices:
        for unit in cutVertices[chosen]:
            
            graph.nodes[chosen]['POP20'] += graph.nodes[unit]['POP20']
            graph.nodes[chosen]['VOTES_DEM'] += graph.nodes[unit]['VOTES_DEM']
            graph.nodes[chosen]['VOTES_REP'] += graph.nodes[unit]['VOTES_REP']
            graph.nodes[chosen]['NHWHITEPOP'] += graph.nodes[unit]['NHWHITEPOP']
            graph.nodes[chosen]['BLACKPOP'] += graph.nodes[unit]['BLACKPOP']
            graph.nodes[chosen]['HISPLATPOP'] += graph.nodes[unit]['HISPLATPOP']
            
            for nb in graph.nodes[unit]['neighbors']:
                graph.nodes[nb]['neighbors'].remove(unit)

            for nb in graph.nodes[unit]['neighbors_aug']:
                graph.nodes[nb]['neighbors_aug'].remove(unit)
                
            graph.remove_node(unit)


    edge_remove = []
    for e in adj:
        if e[0] in cutVertices:
            if e[1] in cutVertices[e[0]]:
                edge_remove.append(e)
                
        elif e[1] in cutVertices:
            if e[0] in cutVertices[e[1]]:
                edge_remove.append(e)
                
        elif e[0] in surrounded and e[1] in surrounded:
            edge_remove.append(e)
            
    for e in edge_remove:
        del adj[e]
    
            
    print('Units merged')
    #print(len(graph.nodes))
    
    return graph,adj,cutVertices_ID
    
    
# Input: Parameter list
# Output: Gerrychain graph object, adjacency dict, cut vertices dict
def make_graph(param):
    
    # Read in shapefile with geopandas
    shapefile_path = param.StateFolder + '/' + param.shapefile[0]
    df = gp.read_file(shapefile_path)

    # Create Graph objects from gerrychain with queen (aug) adjacency
    graph = Graph.from_geodataframe(df,'queen')
    
    # Make sure all GEOIDs are strings, just in case
    for node in graph.nodes:
        temp = str(int(graph.nodes[node]['GEOID20']))
        graph.nodes[node]['GEOID20'] = temp

    # Apply edge weights
    for edge in graph.edges:
        
        if graph.nodes[edge[0]]['GEOID20'][0:5] == graph.nodes[edge[1]]['GEOID20'][0:5]:
            temp = graph.edges[edge]['shared_perim']
            graph.edges[edge]['shared_perim'] = param.edgePenalty*temp
            
        else:
            temp = graph.edges[edge]['shared_perim']
            graph.edges[edge]['shared_perim'] = param.edgeBonus*temp
        
    # Remove any forbidden adjacencies
    for edge in graph.edges:
        if (([graph.nodes[edge[0]]['GEOID20'][0:5],graph.nodes[edge[1]]['GEOID20'][0:5]] in param.forbidden) 
            or ([graph.nodes[edge[1]]['GEOID20'][0:5],graph.nodes[edge[0]]['GEOID20'][0:5]] in param.forbidden)):
            graph.remove_edge(edge[0],edge[1])
        
        
    # Make cleaned adjacency dictionary
    edgesLength = {}

    # Populate adjacency matrix with length of shared segment for normal adjacency and -1 for aug adjacency
    
    # Adjacency to outside
    for node in graph.nodes:
        if graph.nodes[node]['boundary_node']:
            if float(graph.nodes[node]['boundary_perim']) == 0:
                edgesLength[(node,'dummy')] = 0
                edgesLength[('dummy',node)] = 0
            else:
                edgesLength[(node,'dummy')] = float(graph.nodes[node]['boundary_perim'])/1000
                edgesLength[('dummy',node)] = float(graph.nodes[node]['boundary_perim'])/1000

    # Normal adjacency
    for e in graph.edges:
        if float(graph.edges[e]['shared_perim']) > 0:
            edgesLength[(e[0],e[1])] = float(graph.edges[e]['shared_perim'])/1000 
            edgesLength[(e[1],e[0])] = float(graph.edges[e]['shared_perim'])/1000
            
        else:
            edgesLength[(e[0],e[1])] = -1
            edgesLength[(e[1],e[0])] = -1


    
    # Add dummy node
    graph.add_node('dummy')
    graph.nodes['dummy']['GEOID20'] = '0'
    
    
    # Gather a list of neighbors and aug neighbors for every unit
    # Not using built-in Graph.neighbors(node) function because want separate normal and aug neighbors
    # without making two different graph objects and want dummy node
    neighborhoods = {}
    neighborhoods_aug = {}

    for node in graph.nodes:
        neighborhoods[node] = []
        neighborhoods_aug[node] = []
        
    neighborhoods['dummy'] = []
    neighborhoods_aug['dummy'] = []

    for pair in edgesLength:
        if edgesLength[pair] > 0:
            neighborhoods[pair[0]].append(pair[1])
            neighborhoods_aug[pair[0]].append(pair[1])
        elif edgesLength[pair] < 0:
            neighborhoods_aug[pair[0]].append(pair[1])
            
            
    for node in graph.nodes:
        graph.nodes[node]['neighbors'] = neighborhoods[node]
        graph.nodes[node]['neighbors_aug'] = neighborhoods_aug[node]

    
    # Calculate perimeter of each node (does NOT include state perimeter segments)
    perimeters = {}

    for node in graph.nodes:
        perimSum = 0
        for nb in graph.nodes[node]['neighbors']:
            if graph.nodes[nb]['GEOID20'] != '0':
                perimSum += edgesLength[(node,nb)]

        graph.nodes[node]['perimeter'] = perimSum
    
    
    
    graph,edgesLength,cutV = merge_cut_vertices(graph,edgesLength)
      
    return graph,edgesLength,cutV
        

        

#     print(graph.nodes.data())
#     for node in graph.nodes:
#         print(graph.nodes[node]['GEOID20'])

#     print(graph.edges.data())


#     # Can output unit adjacency list
#     outAdj = open('Adjacency.csv','w')
#     writerAdj = csv.writer(outAdj,delimiter=',')

#     writerAdj.writerow(['Unit 1','Unit 2','Shared perim (meters)'])
#     for edge in graph.edges:
#         writerAdj.writerow([str(graph.nodes[edge[0]]['GEOID20']),str(graph.nodes[edge[1]]['GEOID20']),str(graph.edges[edge]['shared_perim'])])

#     for node in graph.nodes:
#         if graph.nodes[node]['boundary_node']:
#             writerAdj.writerow([str(graph.nodes[node]['GEOID20']),'outside',graph.nodes[node]['boundary_perim']])


#     outAdj.close()



    
# Input: .csv file name string for parameter file
# Output: If parameters aren't valid, returns 0
#         If parameters are valid, returns PARAMS object
def read_parameters(filename):
    
    # Create boolean variable to report whether user input is valid
    VALID = True

    # Read in parameters
    parameterFile = open(filename,'r')
    readerParam = csv.reader(parameterFile,delimiter=',')
    parameters = [line for line in readerParam]
    
    param = PARAMS(parameters)

    # Folder of district plan files
    print('Folder with maps: ',param.PlanFolder)

    if os.path.isdir(param.PlanFolder):
        maps = [p for p in os.listdir(param.PlanFolder+'/') if not p.startswith('.')]
        maps.sort()
        param.set_maps(maps)
    else:
        VALID = False
        print('\n-----Folder does not exist-----\n')

    if len(parameters[0]) > 2 and os.path.isdir(param.PlanFolder):
        if parameters[0][2] != '':
            maps = []
            for val in parameters[0][2:]:
                if val == '':
                    continue
                elif os.path.isfile(param.PlanFolder+'/'+val):
                    maps.append(val)
                else:
                    VALID = False
                    print('\n-----File does not exist-----\n')
                    break
                    
            param.set_maps(maps)

            print('Maps:\n',param.maps)

    print('')
    for file in maps:
        if not os.path.isfile(param.PlanFolder+'/'+file):
            print(file)
            VALID = False
            print('\n-----File does not exist-----\n')
            break


    # Folder with state data
    print('Folder with state data: ',param.StateFolder)

    if not os.path.isdir(param.StateFolder):
        VALID = False
        print('\n-----Folder does not exist-----\n')
    else:
        files = [t for t in os.listdir(param.StateFolder+'/') if (not t.startswith('.') and t[-4:] == '.shp')]
        #files.sort()
        param.set_shapefile(files)
        if len(files) != 1:
            VALID = False
            print('\n-----More than 1 or fewer than 1 .shp file in this folder-----\n')


    # Folder for output
    print('Folder for output: ',param.OutputFolder)

    if not os.path.isdir(param.OutputFolder):
        VALID = False
        print('\n-----Folder does not exist-----\n')


    # Number of iterations
    print('Number of iterations: ',param.num_iterations)


    # Objective function
    ValidObjectives = ['pop','perim','cut_edges','eg','eg_shift','mm','pa','cmpttv']

    if param.objective in ValidObjectives:
        print('Objective: ',param.objective)
    else:
        VALID = False
        print('\n-----',param.objective,' is not a valid objective-----\n')


    # Constraints
    print('Constraints:\n',param.constraints)

    ValidConstraints = ['pop','perim','cut_edges','eg','eg_shift','mm','pa','cmpttv','demo_basic','demo','whole']

    for c in param.constraints:
        if c not in ValidConstraints:
            VALID = False
            print('\n-----',c,' is not a valid constraint-----\n')


    # Assign thresholds
    if 'pop' in param.constraints:
        print('Maximum allowed population deviation: ',param.max_pop_dev)

    if 'perim' in param.constraints:
        print('Value multiplied by initial perimeter for perimeter threshold: ',param.perim_mult)

    if 'cut_edges' in param.constraints:
        print('Value multiplied by initial # cut_edges for cut_edges threshold: ',param.cut_edges_mult)

    if 'eg' in param.constraints:
        print('EG threshold: ',param.eg_threshold)

    if 'eg_shift' in param.constraints:
        print('Shifted EG threshold: ',param.seg_threshold)

    if 'mm' in param.constraints:
        print('MM threshold: ',param.mm_threshold)

    if 'pa' in param.constraints:
        print('PA threshold: ',param.pa_threshold)

    if param.objective == 'cmpttv' or 'cmpttv' in param.constraints:
        print('Margin of competitiveness: ',param.cmpttv_margin)


    # Edge weights
    if 'perim' in param.constraints or param.objective == 'perim':
        print('Inter-county perimeter segment bonus: ',param.edgeBonus)
        print('Intra-county perimeter segment penalty: ',param.edgePenalty)


    # Determine if user wants objective convergence
    if param.converge == 'yes':
        param.converge = True
        print('Run until objective values converge')
        if len(parameters[16]) > 2:
            if parameters[16][2] != '':
                param.convergence_threshold = float(parameters[16][2])
                print('Convergence threshold: ',param.convergence_threshold)
            else:
                #VALID = False
                print('\n----- Did not provide convergence threshold, using default of 1 -----\n')
        else:
            #VALID = False
            print('\n----- Did not provide convergence threshold, using default of 1 -----\n')
    elif param.converge == 'no':
        param.converge = False
        print('Run for a fixed number of iterations')
    else:
        VALID = False
        print('\n-----',param.converge,' is not a valid convergence option-----\n')


    # Determine number of repeated runs
    if param.repeated_runs <= 0:
        VALID = False
        print('\n-----',param.repeated_runs,' is not a valid number of replications-----\n')
    else:
        print('Number of replications per map: ',param.repeated_runs)
        
        
    # Determine if time limit
    if param.time_limit_bool == 'yes':
        param.time_limit_bool = True
        print('Time limit:')
        if len(parameters[18]) > 2:
            if parameters[18][2] != '':
                param.time_limit = float(parameters[18][2])
                print('\t',param.time_limit,' seconds')
            else:
                #VALID = False
                print('\n----- Did not provide time limit, using default of 1 second -----\n')
        else:
            #VALID = False
            print('\n----- Did not provide time limit, using default of 1 second -----\n')
    elif param.time_limit_bool == 'no':
        param.time_limit_bool = False
        print('No time limit')
    else:
        VALID = False
        print('\n-----',param.time_limit_bool,' is not a valid time limit option-----\n')

        
    # Local search type
    print('Local search type: ',param.search_type)
    
    if param.search_type == 'SA':
        if len(parameters[19]) > 2:
            if parameters[19][2] != '':
                param.temperature_initial = float(parameters[19][2])
                param.temperature = float(parameters[19][2])
                print('\tSA temperature: ',param.temperature)
            else:
                print('\n----- Did not provide SA temperature, using default of 1.0 -----\n')
                
        if len(parameters[19]) > 3:
            if parameters[19][3] != '':
                param.cooling = float(parameters[19][3])
                print('\tSA cooling factor: ',param.cooling)
            else:
                print('\n----- Did not provide SA cooling factor, using default of 0.999 -----\n')
                
        if len(parameters[19]) > 4:
            if parameters[19][4] != '':
                param.epsilon = float(parameters[19][4])
                print('\tSA epsilon: ',param.epsilon)
            else:
                print('\n----- Did not provide SA epsilon, using default of 0.001 -----\n')
        
        
    # County adjacencies to remove
    print('Adjacency to remove:\n',param.forbidden)
    
    
    # If user input is invalid, stop program
    if not VALID:
        print('\nINVALID USER INPUT -- continuing to run will give an error')
        return 0
    else:
        print('\nUSER INPUT IS VALID -- good to go!')
        return param

    

# Input: PARAMS object, PLAN_VALUES, Gerrychain graph object, adjacency dict
# Output: PLAN_VALUES object, objective value list
def get_district_info(param,dvalues,graph,adj):
    
    obj_list = []
    
    # Count number of counties spanned by each district
    if 'whole' in param.constraints:

        dvalues.distCounties = [[] for p in range(0,dvalues.numDistricts)]
        for node in graph.nodes:
            if graph.nodes[node]['GEOID20'][0:5] not in dvalues.distCounties[graph.nodes[node]['assignment']]:
                dvalues.distCounties[graph.nodes[node]['assignment']].append(graph.nodes[node]['GEOID20'][0:5])
                
                
    # Gather units on the border of each district
    
    # borderNodes: List of all border nodes
    # borderNodesByDist: Records border nodes for each district (used to calculate initial district perimeters & auxH)
    # borderNodesYesNo: For every node, record whether it's a border node 
    #                   (can be adapted to record # of districts node is adjacent to, besides its own)

    for node in graph.nodes:
        dvalues.borderNodesYesNo[node] = 0

    for i in range(0,dvalues.numDistricts):
        dvalues.borderNodesByDist[i] = []

        
    # OLD
        
#     for node in graph.nodes:
#         dist = graph.nodes[node]['assignment']
#         for nb in graph.nodes[node]['neighbors_aug']:
#             otherDist = graph.nodes[nb]['assignment']
#             if otherDist != dist:
#                 dvalues.borderNodesByDist[dist].append(node)
#                 if otherDist != 0 and dist != 0 and adj[(node,nb)] > 0: # Don't want nodes to move to dummy district 0
#                     dvalues.borderNodes.append(node)
#                     dvalues.borderNodesYesNo[node] = 1
#                 break # Exit neighborhood loop if node is already identified as on the border 
#                       #(DON'T need to count units multiple times)
                    
                    
    # New
        
    for node in graph.nodes:
        dist = graph.nodes[node]['assignment']
        for nb in graph.nodes[node]['neighbors_aug']:
            otherDist = graph.nodes[nb]['assignment']
            if otherDist != dist:
                if node not in dvalues.borderNodesByDist[dist]:
                    dvalues.borderNodesByDist[dist].append(node)
                if otherDist != 0 and dist != 0 and adj[(node,nb)] > 0: # Don't want nodes to move to dummy district 0
                    dvalues.borderNodes.append(node)
                    dvalues.borderNodesYesNo[node] = 1
                    break # Exit neighborhood loop if node is already identified as on the border 
                          #(DON'T need to count units multiple times)

                    
    # Make auxiliary district graph for hole check
    for i in range(0,dvalues.numDistricts):
        for j in range(0,dvalues.numDistricts):
            dvalues.auxH[(i,j)] = 0

    for i in range(0,dvalues.numDistricts):
        for b in dvalues.borderNodesByDist[i]:
            for n in graph.nodes[b]['neighbors_aug']:
            #for n in graph.nodes[b]['neighbors']:
                if graph.nodes[n]['assignment'] != i:
                    dvalues.auxH[(i,graph.nodes[n]['assignment'])] += 1
                    
                    
    # Calculate population of each district
    if 'pop' in param.constraints or param.objective == 'pop':
        
        dvalues.distPop = [0 for i in range(0,dvalues.numDistricts)]

        for node in graph.nodes:
            if node != 'dummy':
                dvalues.distPop[graph.nodes[node]['assignment']] += graph.nodes[node]['POP20']

        dvalues.meanDistPop = sum(dvalues.distPop)/((dvalues.numDistricts)-1) #numDistricts-1 bc don't want to include dummy district
        dvalues.distPop[0] = dvalues.meanDistPop # Change dummy districts population to the expected

        # Calculate each district's absolute percent difference from expected population
        dvalues.distPopPercent = [abs(1-(dp)/(dvalues.meanDistPop)) for dp in dvalues.distPop]

        print('Initial max population deviation: ',max(dvalues.distPopPercent))

        if param.objective == 'pop':
            obj_list = [max(dvalues.distPopPercent)]
            
            
            
    # Calculate the fraction of district pop that is white/Black/Lat
    if 'demo_basic' in param.constraints:

        dvalues.fracW = [0.0 for i in range(0,dvalues.numDistricts)]
       
        for node in graph.nodes:
            if node != 'dummy':
                dvalues.fracW[graph.nodes[node]['assignment']] += graph.nodes[node]['NHWHITEPOP']

        for i in range(0,dvalues.numDistricts):
            dvalues.fracW[i] = float(dvalues.fracW[i])/dvalues.distPop[i]

        numPlurW = 0

        for i in range(1,dvalues.numDistricts):
            if dvalues.fracW[i] < 0.5:
                numPlurW += 1

        print('Initial # basic MM districts: ',numPlurW)

        # Still calling it 'numPlurW' to keep same name as 'demo' constraint terms
        # NHWHITEPOP might not actually be a plurality
        dvalues.numPlurW = numPlurW

            
            
    # Calculate the fraction of district pop that is white/Black/Lat
    if 'demo' in param.constraints:

        dvalues.fracW = [0.0 for i in range(0,dvalues.numDistricts)]
        dvalues.fracB = [0.0 for i in range(0,dvalues.numDistricts)]
        dvalues.fracL = [0.0 for i in range(0,dvalues.numDistricts)]

        for node in graph.nodes:
            if node != 'dummy':
                dvalues.fracW[graph.nodes[node]['assignment']] += graph.nodes[node]['NHWHITEPOP']
                dvalues.fracB[graph.nodes[node]['assignment']] += graph.nodes[node]['BLACKPOP']
                dvalues.fracL[graph.nodes[node]['assignment']] += graph.nodes[node]['HISPLATPOP']

        for i in range(0,dvalues.numDistricts):
            dvalues.fracW[i] = float(dvalues.fracW[i])/dvalues.distPop[i]
            dvalues.fracB[i] = float(dvalues.fracB[i])/dvalues.distPop[i]
            dvalues.fracL[i] = float(dvalues.fracL[i])/dvalues.distPop[i]

        numPlurB = 0
        numPlurL = 0
        numPlurW = 0

        for i in range(1,dvalues.numDistricts):
            if dvalues.fracW[i] < 0.5:
                if dvalues.fracB[i] > dvalues.fracW[i] and dvalues.fracB[i] > dvalues.fracL[i]:
                    numPlurB += 1
                elif dvalues.fracL[i] > dvalues.fracW[i] and dvalues.fracL[i] > dvalues.fracB[i]:
                    numPlurL += 1
                else:
                    numPlurW += 1

        print('Initial # plurality-Black MM districts: ',numPlurB)
        print('Initial # plurality-Lat/Hisp MM districts: ',numPlurL)
        print('Initial # plurality-white MM districts: ',numPlurW)
        
        dvalues.numPlurB = numPlurB
        dvalues.numPlurL = numPlurL
        dvalues.numPlurW = numPlurW


            
    # Calculate district perimeters
    if 'perim' in param.constraints or param.objective == 'perim':
        
        dvalues.distPerimeter = [0.0 for i in range(0,dvalues.numDistricts)]
        
        # Calculate state perimeter (remains constant throughout)
        statePerim = 0
        for pair in adj:
            if pair[0] == 'dummy' and adj[pair] > 0: # only need one, since edges are double-recorded
                statePerim += adj[pair]
                
        dvalues.distPerimeter[0] = statePerim # Set dummy district perimeter to state perimeter

        for i in range(1,dvalues.numDistricts):
            for b in dvalues.borderNodesByDist[i]:
                for n in graph.nodes[b]['neighbors']:
                    if graph.nodes[n]['assignment'] != graph.nodes[b]['assignment'] and adj[(n,b)] > 0 and n != 'dummy':
                        dvalues.distPerimeter[i] += adj[(n,b)]

        print('Initial total perimeter: ',sum(dvalues.distPerimeter))
        
        if 'perim' in param.constraints:
            param.perim_threshold = sum(dvalues.distPerimeter) * param.perim_mult
            print('Perimeter threshold: ',param.perim_threshold)

        if param.objective == 'perim':
            obj_list = [sum(dvalues.distPerimeter)]
            
            
    # Calculate # cut edges
    if 'cut_edges' in param.constraints or param.objective == 'cut_edges':
            
        count_cut = 0
        for pair in adj:
            if pair[0] != 'dummy' and pair[1] != 'dummy' and adj[pair] > 0:
                if graph.nodes[pair[0]]['assignment'] != graph.nodes[pair[1]]['assignment']:
                    count_cut += 1

        dvalues.CutEdges = count_cut/2
        print('Initial total cut edges: ',dvalues.CutEdges)

        if 'cut_edges' in param.constraints:
            param.cut_edges_threshold = round(dvalues.CutEdges * param.cut_edges_mult)
            print('Cut edges threshold: ',param.cut_edges_threshold)
            
        if param.objective == 'cut_edges':
            obj_list = [dvalues.CutEdges]
            
            
    # Calculate fraction of dem, and number of dem/rep in every district
    dvalues.fracDem = [0.0 for i in range(0,dvalues.numDistricts)]
    dvalues.numDem = [0.0 for i in range(0,dvalues.numDistricts)]
    dvalues.numRep = [0.0 for i in range(0,dvalues.numDistricts)]

    for node in graph.nodes:
        if node != 'dummy':
            dvalues.numDem[graph.nodes[node]['assignment']] += graph.nodes[node]['VOTES_DEM']
            dvalues.numRep[graph.nodes[node]['assignment']] += graph.nodes[node]['VOTES_REP']

    for i in range(1,dvalues.numDistricts):
        dvalues.fracDem[i] = dvalues.numDem[i]/(dvalues.numDem[i]+dvalues.numRep[i])

    dvalues.totalVotesByDistrict = [dvalues.numDem[i]+dvalues.numRep[i] for i in range(0,len(dvalues.numDem))]
    #totalVotes = sum(totalVotesByDistrict)
    
    
    
    # Calculate initial Efficiency Gap value
    if 'eg' in param.constraints or param.objective == 'eg':

        dvalues.wastedNominal = initialize_wasted(dvalues.numDem[1:],dvalues.numRep[1:])
        EGInitial = update_EG_fast(dvalues.wastedNominal,sum(dvalues.totalVotesByDistrict))
        print('Initial EG: ',EGInitial)

        if param.objective == 'eg':
            obj_list = [EGInitial]
            
            
    # Calculate initial Shifted Efficiency Gap value
    if 'eg_shift' in param.constraints or param.objective == 'eg_shift':

        dvalues.wastedShift = initialize_wastedShift(dvalues.fracDem[1:],dvalues.totalVotesByDistrict[1:])
        EGShiftInitial = update_ShiftedEG_fast(dvalues.wastedShift,sum(dvalues.totalVotesByDistrict),dvalues.numDistricts)
        print('Initial Max EG Shift: ',EGShiftInitial)

        if param.objective == 'eg_shift':
            obj_list = [EGShiftInitial]
    
    
    # Calculate initial Median-Mean value
    if 'mm' in param.constraints or param.objective == 'mm':

        # abs(Median(FD) - Mean(FD))
        MMInitial = abs((np.median(dvalues.fracDem[1:])) - (sum(dvalues.fracDem[1:])/((dvalues.numDistricts)-1)))
        print('Initial MM: ',MMInitial)
            
        if param.objective == 'mm':
            obj_list = [MMInitial]
            
            
    # Calculate initial Partisan Asymmetry value
    if 'pa' in param.constraints or param.objective == 'pa':

        PAInitial = update_PA(dvalues.fracDem[1:])
        print('Initial PA: ',PAInitial)

        if param.objective == 'pa':
            obj_list = [PAInitial]
            
            
    # Calculate initial competitiveness
    if 'cmpttv' in param.constraints or param.objective == 'cmpttv':

        dvalues.compFrac = [abs(2*fd - 1) for fd in dvalues.fracDem[1:]]
        dvalues.compFrac.insert(0,0.0)

        countCmpttv = 0
        for cp in dvalues.compFrac[1:]:
            if cp <= param.cmpttv_margin:
                countCmpttv += 1

        print('Initial # districts within '+str(round(100*param.cmpttv_margin))+'% margin: ',countCmpttv)

        if param.objective == 'cmpttv':
            obj_list = [countCmpttv]
    
            
                    
    return obj_list,dvalues




# Checks contiguity of district distFrom upon removal of node n
# Input: node, N(node) intersect V(district(node)), N_aug(node) intersect V(district(node), Gerrychain graph object
# Output: Bool indicating whether or not distFrom remains contiguous upon removal of node
def checkContig(n,BOTH,BOTH_aug,graph):
    
    # Check contiguity of distFrom upon removal of chosen using a breadth-first search
    isPath = True
    x = BOTH[0] # Recall, nodes is N(n) intersect V(district(n))
    discovered = [x]
    bfsQ = col.deque()
    bfsQ.append(x)
    found = False
    while len(bfsQ) > 0:
        v = bfsQ.popleft()
        vNbhd = set(graph.nodes[v]['neighbors'])
        #vBoth = [x for x in vNbhd if data[x] == From] # For basic BFS
        vBoth = list(vNbhd.intersection(BOTH_aug)) # For aug-nbhd BFS (geo-graph)
        if n in vBoth:
            vBoth.remove(n)

        for w in vBoth:
            if w not in discovered:
                discovered.append(w)
                bfsQ.append(w)

    for y in BOTH:
        if y not in discovered:
            isPath = False
            break
            
    return isPath


# Checks if moving node n from district From to district To creates a hole
# Input: node, district From, district To, Gerrychain graph object, PLAN_VALUES object
# Output: Bool indicating whether or not flipping node creates hole
def checkHole(n,From,To,graph,dvalues):
    
    isPath = True
    disconnect = False

    # Increment if To and otherDist will have an additional node pair
    # Decrement if From and otherDist lose a node pair
    for vert in graph.nodes[n]['neighbors_aug']:
        otherDist = graph.nodes[vert]['assignment']
        if To != otherDist:
            dvalues.auxH[To,otherDist] += 1
            dvalues.auxH[otherDist,To] += 1

        if From != otherDist:
            dvalues.auxH[From,otherDist] -= 1
            dvalues.auxH[otherDist,From] -= 1
            
            # If edge deleted, signal for possible disconnect
            if (dvalues.auxH[From,otherDist] == 0) or (dvalues.auxH[otherDist,From] == 0):
                disconnect = True
                #numDisconnect += 1


    if disconnect:

        # Gather aux neighborhoods
        auxNeighborhoods = [[] for p in range(0,dvalues.numDistricts)]

        for i in range(0,dvalues.numDistricts):
            for j in range(0,dvalues.numDistricts):
                if dvalues.auxH[i,j] > 0:
                    auxNeighborhoods[i].append(j)
                    
        # Remove Too and all Too adjacencies
        auxNeighborhoods[To] = []

        for i in range(0,len(auxNeighborhoods)):
            temp = auxNeighborhoods[i]
            if To in temp:
                temp.remove(To)
                auxNeighborhoods[i] = temp

        # Check that new auxiliary graph H is connected
        isPath = True
        zones = [i for i in range(0,dvalues.numDistricts)]
        zones.remove(To)
        x = zones[0]
        discovered = [x]
        bfsQ = col.deque()
        bfsQ.append(x)
        found = False
        while len(bfsQ) > 0:
            v = bfsQ.popleft()
            auxNbhd = auxNeighborhoods[v]

            for w in auxNbhd:
                if w not in discovered:
                    discovered.append(w)
                    bfsQ.append(w)

        for y in zones:
            if y not in discovered:
                isPath = False
                break


        # Reject chosen node if moving it creates a hole/surrounded zone
        if not isPath:

            # Revert auxH back to previous
            for vert in graph.nodes[n]['neighbors_aug']:
                otherDist = graph.nodes[vert]['assignment']
                if To != otherDist:
                    dvalues.auxH[To,otherDist] -= 1 # Decrement here to undo the increment!
                    dvalues.auxH[otherDist,To] -= 1

                if From != otherDist:
                    dvalues.auxH[From,otherDist] += 1 # Increment here to undo the decrement!
                    dvalues.auxH[otherDist,From] += 1

                    

    return isPath



# Checks if moving node n from district From to district To creates a hole
# Input: node, district From, district To, Gerrychain graph object, PLAN_VALUES object
# Output: Bool indicating whether or not flipping node creates hole
def checkHole_greedy(n,From,To,graph,dvalues):
    
    isPath = True
    disconnect = False

    # Increment if To and otherDist will have an additional node pair
    # Decrement if From and otherDist lose a node pair
    for vert in graph.nodes[n]['neighbors_aug']:
        otherDist = graph.nodes[vert]['assignment']
        if To != otherDist:
            dvalues.auxH[To,otherDist] += 1
            dvalues.auxH[otherDist,To] += 1

        if From != otherDist:
            dvalues.auxH[From,otherDist] -= 1
            dvalues.auxH[otherDist,From] -= 1
            
            # If edge deleted, signal for possible disconnect
            if (dvalues.auxH[From,otherDist] == 0) or (dvalues.auxH[otherDist,From] == 0):
                disconnect = True
                #numDisconnect += 1


    if disconnect:

        # Gather aux neighborhoods
        auxNeighborhoods = [[] for p in range(0,dvalues.numDistricts)]

        for i in range(0,dvalues.numDistricts):
            for j in range(0,dvalues.numDistricts):
                if dvalues.auxH[i,j] > 0:
                    auxNeighborhoods[i].append(j)
                    
        # Remove Too and all Too adjacencies
        auxNeighborhoods[To] = []

        for i in range(0,len(auxNeighborhoods)):
            temp = auxNeighborhoods[i]
            if To in temp:
                temp.remove(To)
                auxNeighborhoods[i] = temp

        # Check that new auxiliary graph H is connected
        isPath = True
        zones = [i for i in range(0,dvalues.numDistricts)]
        zones.remove(To)
        x = zones[0]
        discovered = [x]
        bfsQ = col.deque()
        bfsQ.append(x)
        found = False
        while len(bfsQ) > 0:
            v = bfsQ.popleft()
            auxNbhd = auxNeighborhoods[v]

            for w in auxNbhd:
                if w not in discovered:
                    discovered.append(w)
                    bfsQ.append(w)

        for y in zones:
            if y not in discovered:
                isPath = False
                break



    # Revert auxH back to previous
    for vert in graph.nodes[n]['neighbors_aug']:
        otherDist = graph.nodes[vert]['assignment']
        if To != otherDist:
            dvalues.auxH[To,otherDist] -= 1 # Decrement here to undo the increment!
            dvalues.auxH[otherDist,To] -= 1

        if From != otherDist:
            dvalues.auxH[From,otherDist] += 1 # Increment here to undo the decrement!
            dvalues.auxH[otherDist,From] += 1

                    

    return isPath



# SIMPLE HILL CLIMBING CHECK
# Checks that moving node n from district From to district To satisfies constraints and improves objective
# Input: district From, district To, chosen node, PARAMS object,
#        PLAN_VALUES object, Gerrychain graph object, current objective value
# Output: updated values dict
def FlipCheck_simple(From,To,NODE,param,dvalues,graph,obj_current):
    
    # Use bool to keep track of whether objective/constraints are satisfied
    checks = True
    
    # Record updates in case move is accepted
    updatedValues = {}
    
    # Determine intersection of N(chosen node)/N_aug(chosen node) and V(From)
    both = [n for n in graph.nodes[NODE]['neighbors'] if graph.nodes[n]['assignment'] == From]
    both_aug = [n for n in graph.nodes[NODE]['neighbors_aug'] if graph.nodes[n]['assignment'] == From]
    
    # Reject move if node is the only unit in its district
    if len(both) == 0:
        checks = False
        updatedValues['why'] = 'empty'

    # Check pop objective/constraint, if applicable
    if checks:
        if 'pop' in param.constraints or param.objective == 'pop':

            # Determine new district populations
            newPopFrom = dvalues.distPop[From] - graph.nodes[NODE]['POP20']
            newPopTo = dvalues.distPop[To] + graph.nodes[NODE]['POP20']

            percentFrom = abs(1-(newPopFrom/dvalues.meanDistPop))
            percentTo = abs(1-(newPopTo/dvalues.meanDistPop))

            dev = max(percentFrom,percentTo)

            if param.objective == 'pop':
                if dev > max(dvalues.distPopPercent[From],dvalues.distPopPercent[To]):
                    checks = False
                    updatedValues['why'] = 'pop'
                    
                else:
                    updatedValues['pop'] = [newPopFrom,newPopTo]
                        

            if 'pop' in param.constraints:
                if dev > param.max_pop_dev:
                    checks = False
                    updatedValues['why'] = 'pop'
                else:
                    updatedValues['pop'] = [newPopFrom,newPopTo]


    # If previous checks pass, check perim objective/constraint, if applicable
    if checks:
        if 'perim' in param.constraints or param.objective == 'perim':
            
            perimFrom = 0
            perimTo = 0
            for nb in graph.nodes[NODE]['neighbors']:
                if graph.nodes[nb]['assignment'] == From:
                    perimFrom += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimFrom += adj[(NODE,nb)]
                elif graph.nodes[nb]['assignment'] == To:
                    perimTo += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimTo += adj[(NODE,nb)]

            newPerimFrom = dvalues.distPerimeter[From] + (2*perimFrom - graph.nodes[NODE]['perimeter'])
            newPerimTo = dvalues.distPerimeter[To] + (graph.nodes[NODE]['perimeter'] - 2*perimTo)

            newPerimeter = newPerimFrom + newPerimTo
            #updatedValues['perim'] = [newPerimFrom,newPerimTo]

            if param.objective == 'perim':
                if newPerimeter > (dvalues.distPerimeter[From]+dvalues.distPerimeter[To]):
                    checks = False
                    updatedValues['why'] = 'perim'                  
                else:
                     updatedValues['perim'] = [newPerimFrom,newPerimTo]

            if 'perim' in param.constraints:
                if (newPerimeter + sum(dvalues.distPerimeter) - dvalues.distPerimeter[From]
                                                 - dvalues.distPerimeter[To]) > param.perim_threshold:
                
                    checks = False
                    updatedValues['why'] = 'perim'
                else:
                    updatedValues['perim'] = [newPerimFrom,newPerimTo]
            
            
    # If previous checks pass, check cut edges objective/constraint, if applicable
    if checks:
        if 'cut_edges' in param.constraints or param.objective == 'cut_edges':
            
            cut_orig = []
            cut_new = []
            for nb in graph.nodes[NODE]['neighbors']:
                for u in graph.nodes[nb]['neighbors']:
                    if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_orig:
                        cut_orig.append((nb,u))
                        cut_orig.append((u,nb))
                    if nb == NODE:
                        if To != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if u == NODE:
                        if To != graph.nodes[nb]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if (nb != NODE) and (u != NODE):
                        if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))

            cut_orig_val = len(cut_orig)/2
            cut_new_val = len(cut_new)/2

            newCutEdges = dvalues.CutEdges - cut_orig_val + cut_new_val

            if param.objective == 'cut_edges':
                if newCutEdges > dvalues.CutEdges:
                    checks = False
                    updatedValues['why'] = 'cut_edges'            
                else:
                    updatedValues['cut_edges'] = newCutEdges

            if 'cut_edges' in param.constraints:
                if newCutEdges > param.cut_edges_threshold:
                    checks = False
                    updatedValues['why'] = 'cut_edges'
                else:
                    updatedValues['cut_edges'] = newCutEdges
                
                
    # If previous checks pass, check demo constraint, if applicable
    if checks:
        if 'demo' in param.constraints:
            
            # Calculate new % white, Black, Lat/Hisp
            newFracWFrom = ((dvalues.fracW[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracWTo = ((dvalues.fracW[To]*dvalues.distPop[To])
                          + graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])
            
            newFracBFrom = ((dvalues.fracB[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['BLACKPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracBTo = ((dvalues.fracB[To]*dvalues.distPop[To]) 
                          + graph.nodes[NODE]['BLACKPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])
            
            newFracLFrom = ((dvalues.fracL[From]*dvalues.distPop[From])
                            - graph.nodes[NODE]['HISPLATPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracLTo = ((dvalues.fracL[To]*dvalues.distPop[To]) 
                          + graph.nodes[NODE]['HISPLATPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            countB = 0
            countL = 0
            countW = 0

            if newFracWFrom < 0.5:
                if newFracBFrom > max(newFracWFrom,newFracLFrom):
                    countB += 1
                elif newFracLFrom > max(newFracWFrom,newFracBFrom):
                    countL += 1
                else:
                    countW += 1

            if newFracWTo < 0.5:
                if newFracBTo > max(newFracWTo,newFracLTo):
                    countB += 1
                elif newFracLTo > max(newFracWTo,newFracBTo):
                    countL += 1
                else:
                    countW += 1
                    
                    
            numW = 0
            numB = 0
            numL = 0

            if dvalues.fracW[From] < 0.5:
                if dvalues.fracB[From] > max(dvalues.fracW[From],dvalues.fracL[From]):
                    numB += 1
                elif dvalues.fracL[From] > max(dvalues.fracW[From],dvalues.fracB[From]):
                    numL += 1
                else:
                    numW += 1

            if dvalues.fracW[To] < 0.5:
                if dvalues.fracB[To] > max(dvalues.fracW[To],dvalues.fracL[To]):
                    numB += 1
                elif dvalues.fracL[To] > max(dvalues.fracW[To],dvalues.fracB[To]):
                    numL += 1
                else:
                    numW += 1
                    

            if countB != numB or countL != numL or countW != numW:
                checks = False
                updatedValues['why'] = 'demo'
            else:
                updatedValues['fracW'] = [newFracWFrom,newFracWTo]
                updatedValues['fracB'] = [newFracBFrom,newFracBTo]
                updatedValues['fracL'] = [newFracLFrom,newFracLTo]
                
                
    # If previous checks pass, check demo_basic constraint, if applicable
    if checks:
        if 'demo_basic' in param.constraints:
            
            # Calculate new % white, Black, Lat/Hisp
            newFracWFrom = ((dvalues.fracW[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracWTo = ((dvalues.fracW[To]*dvalues.distPop[To])
                          + graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])
            
            countW = 0

            if newFracWFrom < 0.5:
                countW += 1

            if newFracWTo < 0.5:
                countW += 1
                
                
            numW = 0

            if dvalues.fracW[From] < 0.5:
                numW += 1

            if dvalues.fracW[distTo] < 0.5:
                numW += 1
                
            if countW != numW:
                checks = False
                updatedValues['why'] = 'demo'
            else:
                updatedValues['fracW'] = [newFracWFrom,newFracWTo]
                
                
    # If previous checks pass, check whole constraint, if applicable, 
    # and only if To is already wholly contained within 1 county
    if checks:
        if ('whole' in param.constraints) and (len(dvalues.distCounties[To]) == 1):
            
            if graph.nodes[NODE]['GEOID20'][0:5] not in dvalues.distCounties[To]:
                checks = False
                updatedValues['why'] = 'whole'
                    
                    
    # If previous checks pass, check contiguity -- mandatory!
    if checks:
        checks = checkContig(NODE,both,both_aug,graph)
        if not checks:
            updatedValues['why'] = 'contig'
        
        
    # Update votes
    if checks:
        
        newNumDemFrom = dvalues.numDem[From] - graph.nodes[NODE]['VOTES_DEM']
        newNumRepFrom = dvalues.numRep[From] - graph.nodes[NODE]['VOTES_REP']
        newNumDemTo = dvalues.numDem[To] + graph.nodes[NODE]['VOTES_DEM']
        newNumRepTo = dvalues.numRep[To] + graph.nodes[NODE]['VOTES_REP']

        newTVFrom = newNumDemFrom + newNumRepFrom
        newTVTo = newNumDemTo + newNumRepTo

        newFDFrom = newNumDemFrom/newTVFrom
        newFDTo = newNumDemTo/newTVTo

        updatedValues['ND'] = [newNumDemFrom,newNumDemTo]
        updatedValues['NR'] = [newNumRepFrom,newNumRepTo]
        
        
    # If previous checks pass, check cmpttv objective/constraint, if applicable
    if checks:
        if 'cmpttv' in param.constraints or param.objective == 'cmpttv':

            # Calculate margins of victory in both districts
            if newTVFrom != 0.0 and newTVTo != 0.0:
                newCmpttvFrom = abs(newNumDemFrom-newNumRepFrom)/newTVFrom
                newCmpttvTo = abs(newNumDemTo-newNumRepTo)/newTVTo
            else:
                newCmpttvFrom = 1.5
                newCmpttvTo = 1.5

            if 'cmpttv' in param.constraints:

                # Check whether same number of competitive districts
                if dvalues.compFrac[From] <= param.cmpttv_margin and dvalues.compFrac[To] <= param.cmpttv_margin:
                    if newCmpttvFrom > param.cmpttv_margin or newCmpttvTo > param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

                elif dvalues.compFrac[From] <= param.cmpttv_margin or dvalues.compFrac[To] <= param.cmpttv_margin:
                    if newCmpttvFrom > param.cmpttv_margin and newCmpttvTo > param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                    elif newCmpttvFrom <= param.cmpttv_margin and newCmpttvTo <= param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                        
                elif dvalues.compFrac[From] > param.cmpttv_margin and dvalues.compFrac[To] > param.cmpttv_margin:
                    if newCmpttvFrom <= param.cmpttv_margin or newCmpttvTo <= param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

            else:
                
                # Check that number of competitive districts did not decrease
                if dvalues.compFrac[From] <= param.cmpttv_margin and dvalues.compFrac[To] <= param.cmpttv_margin:
                    if newCmpttvFrom > param.cmpttv_margin or newCmpttvTo > param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

                elif dvalues.compFrac[From] <= param.cmpttv_margin or dvalues.compFrac[To] <= param.cmpttv_margin:
                    if newCmpttvFrom > param.cmpttv_margin and newCmpttvTo > param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                                                                                         
                    
    # If previous checks pass, check mm objective/constraint, if applicable
    if checks:
        if 'mm' in param.constraints or param.objective == 'mm':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo

                mm = abs((np.median(newFD[1:])) - (sum(newFD[1:])/((dvalues.numDistricts)-1)))

            else:
                mm = 1.5
                

            if 'mm' in param.constraints and mm > param.mm_threshold:
                checks = False
                updatedValues['why'] = 'mm'
            
            if param.objective == 'mm':
                if mm > obj_current:
                    checks = False
                    updatedValues['why'] = 'mm'
                else:
                    updatedValues['mm'] = mm
        
        
    # If previous checks pass, check eg objective/constraint, if applicable
    if checks:
        if 'eg' in param.constraints or param.objective == 'eg':

            # (From,To,FDFrom,FDTo,TVFrom,TVTo,num,waste,total)
            if newTVFrom != 0.0 and newTVTo != 0.0:
                eg,wFrom,wTo = update_EG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedNominal,sum(dvalues.totalVotesByDistrict))
            else:
                eg = 1.5

                
            if 'eg' in param.constraints:
                if eg > param.eg_threshold:
                    checks = False
                    updatedValues['why'] = 'eg'
                else:
                    updatedValues['eg'] = [eg,wFrom,wTo]
            
            if param.objective == 'eg':
                if eg > obj_current:
                    checks = False
                    updatedValues['why'] = 'eg'                                    
                else:
                    updatedValues['eg'] = [eg,wFrom,wTo]
                         
    
    # If previous checks pass, check eg_shift objective/constraint, if applicable
    if checks:
        if 'eg_shift' in param.constraints or param.objective == 'eg_shift':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                seg,wFrom,wTo = update_ShiftedEG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedShift,sum(dvalues.totalVotesByDistrict))
            else:
                seg = 1.5
                
                
            if 'eg_shift' in param.constraints:
                if seg > param.seg_threshold:
                    checks = False
                    updatedValues['why'] = 'eg_shift'
                else:
                    updatedValues['eg_shift'] = [seg,wFrom,wTo]
            
            if param.objective == 'eg_shift':
                if seg > obj_current:
                    checks = False
                    updatedValues['why'] = 'eg_shift'
                else:
                    updatedValues['eg_shift'] = [seg,wFrom,wTo]
                
                
    # If previous checks pass, check pa objective/constraint, if applicable
    if checks:
        if 'pa' in param.constraints or param.objective == 'pa':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo
                pa = update_PA(newFD[1:])
            else:
                pa = 1.5
                
                
            if 'pa' in param.constraints and pa > param.pa_threshold:
                checks = False
                updatedValues['why'] = 'pa'
                
            if param.objective == 'pa':
                if pa > obj_current:
                    checks = False
                    updatedValues['why'] = 'pa'
                else:
                    updatedValues['pa'] = pa
                
                
    # If previous checks pass, check for hole -- mandatory!
    if checks:
        checks = checkHole(NODE,From,To,graph,dvalues)
        if not checks:
            updatedValues['why'] = 'hole'

        
    # Return values
    if checks:
        updatedValues['accept'] = True
    else:
        updatedValues['accept'] = False
        
    return updatedValues



# SIMULATED ANNEALING CHECK
# Checks that moving node n from district From to district To satisfies constraints and improves objective
# Input: district From, district To, chosen node, PARAMS object,
#        PLAN_VALUES object, Gerrychain graph object, current objective value
# Output: updated values dict
def FlipCheck_SA(From,To,NODE,param,dvalues,graph,obj_current):
    
    # Use bool to keep track of whether objective/constraints are satisfied
    checks = True
    
    # Record updates in case move is accepted
    updatedValues = {}
    
    # Determine intersection of N(chosen node)/N_aug(chosen node) and V(From)
    both = [n for n in graph.nodes[NODE]['neighbors'] if graph.nodes[n]['assignment'] == From]
    both_aug = [n for n in graph.nodes[NODE]['neighbors_aug'] if graph.nodes[n]['assignment'] == From]
    
    # Reject move if node is the only unit in its district
    if len(both) == 0:
        checks = False
        updatedValues['why'] = 'empty'

    # Check pop objective/constraint, if applicable
    if checks:
        if 'pop' in param.constraints or param.objective == 'pop':

            # Determine new district populations
            newPopFrom = dvalues.distPop[From] - graph.nodes[NODE]['POP20']
            newPopTo = dvalues.distPop[To] + graph.nodes[NODE]['POP20']

            percentFrom = abs(1-(newPopFrom/dvalues.meanDistPop))
            percentTo = abs(1-(newPopTo/dvalues.meanDistPop))

            dev = max(percentFrom,percentTo)

            if param.objective == 'pop':
                if dev > max(dvalues.distPopPercent[From],dvalues.distPopPercent[To]):
                    
                    SA_prob = np.exp(-abs(max(dev,obj_current)-obj_current)/param.temperature)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'pop'
                    else:
                        updatedValues['pop'] = [newPopFrom,newPopTo]
                        updatedValues['worsen'] = True

                else:
                    updatedValues['pop'] = [newPopFrom,newPopTo]
                    updatedValues['worsen'] = False


            if 'pop' in param.constraints:
                if dev > param.max_pop_dev:
                    checks = False
                    updatedValues['why'] = 'pop'
                else:
                    updatedValues['pop'] = [newPopFrom,newPopTo]


    # If previous checks pass, check perim objective/constraint, if applicable
    if checks:
        if 'perim' in param.constraints or param.objective == 'perim':

            perimFrom = 0
            perimTo = 0
            for nb in graph.nodes[NODE]['neighbors']:
                if graph.nodes[nb]['assignment'] == From:
                    perimFrom += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimFrom += adj[(NODE,nb)]
                elif graph.nodes[nb]['assignment'] == To:
                    perimTo += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimTo += adj[(NODE,nb)]

            newPerimFrom = dvalues.distPerimeter[From] + (2*perimFrom - graph.nodes[NODE]['perimeter'])
            newPerimTo = dvalues.distPerimeter[To] + (graph.nodes[NODE]['perimeter'] - 2*perimTo)

            newPerimeter = newPerimFrom + newPerimTo

            if param.objective == 'perim':
                if newPerimeter > (dvalues.distPerimeter[From]+dvalues.distPerimeter[To]):
                    
                    SA_prob = np.exp(-abs((newPerimeter + sum(dvalues.distPerimeter) - dvalues.distPerimeter[From]
                                                 - dvalues.distPerimeter[To])-obj_current)/param.temperature)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'perim'
                    else:
                        updatedValues['perim'] = [newPerimFrom,newPerimTo]
                        updatedValues['worsen'] = True
                        
                else:
                    updatedValues['perim'] = [newPerimFrom,newPerimTo]
                    updatedValues['worsen'] = False

            if 'perim' in param.constraints:
                if (newPerimeter + sum(dvalues.distPerimeter) - dvalues.distPerimeter[From]
                                                 - dvalues.distPerimeter[To]) > param.perim_threshold:

                    checks = False
                    updatedValues['why'] = 'perim'
                else:
                    updatedValues['perim'] = [newPerimFrom,newPerimTo]

            
    # If previous checks pass, check cut edges objective/constraint, if applicable
    if checks:
        if 'cut_edges' in param.constraints or param.objective == 'cut_edges':

            cut_orig = []
            cut_new = []
            for nb in graph.nodes[NODE]['neighbors']:
                for u in graph.nodes[nb]['neighbors']:
                    if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_orig:
                        cut_orig.append((nb,u))
                        cut_orig.append((u,nb))
                    if nb == NODE:
                        if To != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if u == NODE:
                        if To != graph.nodes[nb]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if (nb != NODE) and (u != NODE):
                        if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))

            cut_orig_val = len(cut_orig)/2
            cut_new_val = len(cut_new)/2

            newCutEdges = dvalues.CutEdges - cut_orig_val + cut_new_val
            
            if param.objective == 'cut_edges':
                if newCutEdges > dvalues.CutEdges:
                    
                    SA_prob = np.exp(-abs(newCutEdges-obj_current)/param.temperature)
                    #print(SA_prob)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'cut_edges'
                    else:
                        updatedValues['cut_edges'] = newCutEdges
                        updatedValues['worsen'] = True
                    
                else:
                    updatedValues['cut_edges'] = newCutEdges
                    updatedValues['worsen'] = False

            if 'cut_edges' in param.constraints:
                if newCutEdges > param.cut_edges_threshold:
                    checks = False
                    updatedValues['why'] = 'cut_edges'
                else:
                    updatedValues['cut_edges'] = newCutEdges


    # If previous checks pass, check demo constraint, if applicable
    if checks:
        if 'demo' in param.constraints:

            # Calculate new % white, Black, Lat/Hisp
            newFracWFrom = ((dvalues.fracW[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracWTo = ((dvalues.fracW[To]*dvalues.distPop[To])
                          + graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            newFracBFrom = ((dvalues.fracB[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['BLACKPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracBTo = ((dvalues.fracB[To]*dvalues.distPop[To]) 
                          + graph.nodes[NODE]['BLACKPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            newFracLFrom = ((dvalues.fracL[From]*dvalues.distPop[From])
                            - graph.nodes[NODE]['HISPLATPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracLTo = ((dvalues.fracL[To]*dvalues.distPop[To]) 
                          + graph.nodes[NODE]['HISPLATPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            countB = 0
            countL = 0
            countW = 0

            if newFracWFrom < 0.5:
                if newFracBFrom > max(newFracWFrom,newFracLFrom):
                    countB += 1
                elif newFracLFrom > max(newFracWFrom,newFracBFrom):
                    countL += 1
                else:
                    countW += 1

            if newFracWTo < 0.5:
                if newFracBTo > max(newFracWTo,newFracLTo):
                    countB += 1
                elif newFracLTo > max(newFracWTo,newFracBTo):
                    countL += 1
                else:
                    countW += 1


            numW = 0
            numB = 0
            numL = 0

            if dvalues.fracW[From] < 0.5:
                if dvalues.fracB[From] > max(dvalues.fracW[From],dvalues.fracL[From]):
                    numB += 1
                elif dvalues.fracL[From] > max(dvalues.fracW[From],dvalues.fracB[From]):
                    numL += 1
                else:
                    numW += 1

            if dvalues.fracW[To] < 0.5:
                if dvalues.fracB[To] > max(dvalues.fracW[To],dvalues.fracL[To]):
                    numB += 1
                elif dvalues.fracL[To] > max(dvalues.fracW[To],dvalues.fracB[To]):
                    numL += 1
                else:
                    numW += 1


            if countB != numB or countL != numL or countW != numW:
                checks = False
                updatedValues['why'] = 'demo'
            else:
                updatedValues['fracW'] = [newFracWFrom,newFracWTo]
                updatedValues['fracB'] = [newFracBFrom,newFracBTo]
                updatedValues['fracL'] = [newFracLFrom,newFracLTo]

                
    # If previous checks pass, check demo_basic constraint, if applicable
    if checks:
        if 'demo_basic' in param.constraints:

            # Calculate new % white, Black, Lat/Hisp
            newFracWFrom = ((dvalues.fracW[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracWTo = ((dvalues.fracW[To]*dvalues.distPop[To])
                          + graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            countW = 0

            if newFracWFrom < 0.5:
                countW += 1

            if newFracWTo < 0.5:
                countW += 1


            numW = 0

            if dvalues.fracW[From] < 0.5:
                numW += 1

            if dvalues.fracW[distTo] < 0.5:
                numW += 1


            if countW != numW:
                checks = False
                updatedValues['why'] = 'demo'
            else:
                updatedValues['fracW'] = [newFracWFrom,newFracWTo]

                
    # If previous checks pass, check whole constraint, if applicable, 
    # and only if To is already wholly contained within 1 county
    if checks:
        if ('whole' in param.constraints) and (len(dvalues.distCounties[To]) == 1):

            if graph.nodes[NODE]['GEOID20'][0:5] not in dvalues.distCounties[To]:
                checks = False
                updatedValues['why'] = 'whole'
                    
                    
    # If previous checks pass, check contiguity -- mandatory!
    if checks:
        checks = checkContig(NODE,both,both_aug,graph)
        if not checks:
            updatedValues['why'] = 'contig'
        
        
    # Update votes
    if checks:
        
        newNumDemFrom = dvalues.numDem[From] - graph.nodes[NODE]['VOTES_DEM']
        newNumRepFrom = dvalues.numRep[From] - graph.nodes[NODE]['VOTES_REP']
        newNumDemTo = dvalues.numDem[To] + graph.nodes[NODE]['VOTES_DEM']
        newNumRepTo = dvalues.numRep[To] + graph.nodes[NODE]['VOTES_REP']

        newTVFrom = newNumDemFrom + newNumRepFrom
        newTVTo = newNumDemTo + newNumRepTo

        newFDFrom = newNumDemFrom/newTVFrom
        newFDTo = newNumDemTo/newTVTo

        updatedValues['ND'] = [newNumDemFrom,newNumDemTo]
        updatedValues['NR'] = [newNumRepFrom,newNumRepTo]
        
        
    # If previous checks pass, check cmpttv objective/constraint, if applicable
    if checks:
        if 'cmpttv' in param.constraints or param.objective == 'cmpttv':

            # Calculate margins of victory in both districts
            if newTVFrom != 0.0 and newTVTo != 0.0:
                newCmpttvFrom = abs(newNumDemFrom-newNumRepFrom)/newTVFrom
                newCmpttvTo = abs(newNumDemTo-newNumRepTo)/newTVTo
            else:
                newCmpttvFrom = 1.5
                newCmpttvTo = 1.5
                

            if 'cmpttv' in param.constraints:

                # Check whether same number of competitive districts
                if dvalues.compFrac[From] <= param.cmpttv_margin and dvalues.compFrac[To] <= param.cmpttv_margin:
                    if newCmpttvFrom > param.cmpttv_margin or newCmpttvTo > param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

                elif dvalues.compFrac[From] <= param.cmpttv_margin or dvalues.compFrac[To] <= param.cmpttv_margin:
                    if newCmpttvFrom > param.cmpttv_margin and newCmpttvTo > param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                    elif newCmpttvFrom <= param.cmpttv_margin and newCmpttvTo <= param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

                elif dvalues.compFrac[From] > param.cmpttv_margin and dvalues.compFrac[To] > param.cmpttv_margin:
                    if newCmpttvFrom <= param.cmpttv_margin or newCmpttvTo <= param.cmpttv_margin:
                        checks = False
                        updatedValues['why'] = 'cmpttv'

            else:
                
                newNumCmpttv = (obj_current - (dvalues.compFrac[From] <= param.cmpttv_margin) 
                                - (dvalues.compFrac[To] <= param.cmpttv_margin) 
                                + (newCmpttvFrom <= param.cmpttv_margin)
                                + (newCmpttvTo <= param.cmpttv_margin))

                # Check that number of competitive districts did not decrease
                if newNumCmpttv < obj_current:
                    
                    SA_prob = np.exp(-abs(newNumCmpttv-obj_current)/param.temperature)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'cmpttv'
                    else:
                        updatedValues['worsen'] = True
                        
                else:
                    updatedValues['worsen'] = False
                
                
#                 if dvalues.compFrac[From] <= param.cmpttv_margin and dvalues.compFrac[To] <= param.cmpttv_margin:
#                     if newCmpttvFrom > param.cmpttv_margin or newCmpttvTo > param.cmpttv_margin:
#                         checks = False
#                         updatedValues['why'] = 'cmpttv'

#                 elif dvalues.compFrac[From] <= param.cmpttv_margin or dvalues.compFrac[To] <= param.cmpttv_margin:
#                     if newCmpttvFrom > param.cmpttv_margin and newCmpttvTo > param.cmpttv_margin:
#                         checks = False
#                         updatedValues['why'] = 'cmpttv'

                    
    # If previous checks pass, check mm objective/constraint, if applicable
    if checks:
        if 'mm' in param.constraints or param.objective == 'mm':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo

                mm = abs((np.median(newFD[1:])) - (sum(newFD[1:])/((dvalues.numDistricts)-1)))

            else:
                mm = 1.5


            if 'mm' in param.constraints and mm > param.mm_threshold:
                checks = False
                updatedValues['why'] = 'mm'

            if param.objective == 'mm':
                if mm > obj_current:
                    
                    SA_prob = np.exp(-abs(mm-obj_current)/param.temperature)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'mm'
                    else:
                        updatedValues['mm'] = mm
                        updatedValues['worsen'] = True
                    
                else:
                    updatedValues['mm'] = mm
                    updatedValues['worsen'] = False
                

        
    # If previous checks pass, check eg objective/constraint, if applicable
    if checks:
        if 'eg' in param.constraints or param.objective == 'eg':

            # (From,To,FDFrom,FDTo,TVFrom,TVTo,num,waste,total)
            if newTVFrom != 0.0 and newTVTo != 0.0:
                eg,wFrom,wTo = update_EG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedNominal,sum(dvalues.totalVotesByDistrict))
            else:
                eg = 1.5


            if 'eg' in param.constraints:
                if eg > param.eg_threshold:
                    checks = False
                    updatedValues['why'] = 'eg'
                else:
                    updatedValues['eg'] = [eg,wFrom,wTo]

            if param.objective == 'eg':
                if eg > obj_current:
                    
                    SA_prob = np.exp(-abs(eg-obj_current)/param.temperature)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'eg'
                    else:
                        updatedValues['eg'] = [eg,wFrom,wTo]
                        updatedValues['worsen'] = True
                    
                else:
                    updatedValues['eg'] = [eg,wFrom,wTo]
                    updatedValues['worsen'] = False

    
    # If previous checks pass, check eg_shift objective/constraint, if applicable
    if checks:
        if 'eg_shift' in param.constraints or param.objective == 'eg_shift':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                seg,wFrom,wTo = update_ShiftedEG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedShift,sum(dvalues.totalVotesByDistrict))
            else:
                seg = 1.5


            if 'eg_shift' in param.constraints:
                if seg > param.seg_threshold:
                    checks = False
                    updatedValues['why'] = 'eg_shift'
                else:
                    updatedValues['eg_shift'] = [seg,wFrom,wTo]

            if param.objective == 'eg_shift':
                if seg > obj_current:
                    
                    SA_prob = np.exp(-abs(seg-obj_current)/param.temperature)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'eg_shift'
                    else:
                        updatedValues['eg_shift'] = [seg,wFrom,wTo]
                        updatedValues['worsen'] = True
                    
                else:
                    updatedValues['eg_shift'] = [seg,wFrom,wTo]
                    updatedValues['worsen'] = False
                
                
    # If previous checks pass, check pa objective/constraint, if applicable
    if checks:
        if 'pa' in param.constraints or param.objective == 'pa':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo
                pa = update_PA(newFD[1:])
            else:
                pa = 1.5


            if 'pa' in param.constraints and pa > param.pa_threshold:
                checks = False
                updatedValues['why'] = 'pa'

            if param.objective == 'pa':
                if pa > obj_current:
                    
                    SA_prob = np.exp(-abs(pa-obj_current)/param.temperature)
                    if random.random() >= SA_prob:
                        checks = False
                        updatedValues['why'] = 'pa'
                    else:
                        updatedValues['pa'] = pa
                        updatedValues['worsen'] = True
                    
                else:
                    updatedValues['pa'] = pa
                    updatedValues['worsen'] = False
                

                
    # If previous checks pass, check for hole -- mandatory!
    if checks:
        checks = checkHole(NODE,From,To,graph,dvalues)
        if not checks:
            updatedValues['why'] = 'hole'

        
    # Return values
    if checks:
        updatedValues['accept'] = True
    else:
        updatedValues['accept'] = False
        
    return updatedValues



# GREEDY CHECK - feasibility
# Checks that moving node n from district From to district To satisfies constraints
# Input: district From, district To, chosen node, PARAMS object,
#        PLAN_VALUES object, Gerrychain graph object, current objective value
# Output: updated values dict
def FlipCheck_greedy_feasible(From,To,NODE,param,dvalues,graph):
    
    # Use bool to keep track of whether objective/constraints are satisfied
    checks = True
    
    # Record updates in case move is accepted
    updatedValues = {}
    
    # Determine intersection of N(chosen node)/N_aug(chosen node) and V(From)
    both = [n for n in graph.nodes[NODE]['neighbors'] if graph.nodes[n]['assignment'] == From]
    both_aug = [n for n in graph.nodes[NODE]['neighbors_aug'] if graph.nodes[n]['assignment'] == From]
    
    # Reject move if node is the only unit in its district
    if len(both) == 0:
        checks = False
        updatedValues['why'] = 'empty'

    # Check pop objective/constraint, if applicable
    if checks:
        if 'pop' in param.constraints:

            # Determine new district populations
            newPopFrom = dvalues.distPop[From] - graph.nodes[NODE]['POP20']
            newPopTo = dvalues.distPop[To] + graph.nodes[NODE]['POP20']

            percentFrom = abs(1-(newPopFrom/dvalues.meanDistPop))
            percentTo = abs(1-(newPopTo/dvalues.meanDistPop))

            dev = max(percentFrom,percentTo)

            if dev > param.max_pop_dev:
                checks = False
                updatedValues['why'] = 'pop'
            else:
                updatedValues['pop'] = [newPopFrom,newPopTo]


    # If previous checks pass, check perim objective/constraint, if applicable
    if checks:
        if 'perim' in param.constraints:

            perimFrom = 0
            perimTo = 0
            for nb in graph.nodes[NODE]['neighbors']:
                if graph.nodes[nb]['assignment'] == From:
                    perimFrom += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimFrom += adj[(NODE,nb)]
                elif graph.nodes[nb]['assignment'] == To:
                    perimTo += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimTo += adj[(NODE,nb)]

            newPerimFrom = dvalues.distPerimeter[From] + (2*perimFrom - graph.nodes[NODE]['perimeter'])
            newPerimTo = dvalues.distPerimeter[To] + (graph.nodes[NODE]['perimeter'] - 2*perimTo)

            newPerimeter = newPerimFrom + newPerimTo

            if (newPerimeter + sum(dvalues.distPerimeter) - dvalues.distPerimeter[From]
                                             - dvalues.distPerimeter[To]) > param.perim_threshold:

                checks = False
                updatedValues['why'] = 'perim'
            else:
                updatedValues['perim'] = [newPerimFrom,newPerimTo]

            
    # If previous checks pass, check cut edges objective/constraint, if applicable
    if checks:
        if 'cut_edges' in param.constraints:

            cut_orig = []
            cut_new = []
            for nb in graph.nodes[NODE]['neighbors']:
                for u in graph.nodes[nb]['neighbors']:
                    if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_orig:
                        cut_orig.append((nb,u))
                        cut_orig.append((u,nb))
                    if nb == NODE:
                        if To != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if u == NODE:
                        if To != graph.nodes[nb]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if (nb != NODE) and (u != NODE):
                        if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))

            cut_orig_val = len(cut_orig)/2
            cut_new_val = len(cut_new)/2

            newCutEdges = dvalues.CutEdges - cut_orig_val + cut_new_val
            
            if newCutEdges > param.cut_edges_threshold:
                checks = False
                updatedValues['why'] = 'cut_edges'
            else:
                updatedValues['cut_edges'] = newCutEdges


    # If previous checks pass, check demo constraint, if applicable
    if checks:
        if 'demo' in param.constraints:

            # Calculate new % white, Black, Lat/Hisp
            newFracWFrom = ((dvalues.fracW[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracWTo = ((dvalues.fracW[To]*dvalues.distPop[To])
                          + graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            newFracBFrom = ((dvalues.fracB[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['BLACKPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracBTo = ((dvalues.fracB[To]*dvalues.distPop[To]) 
                          + graph.nodes[NODE]['BLACKPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            newFracLFrom = ((dvalues.fracL[From]*dvalues.distPop[From])
                            - graph.nodes[NODE]['HISPLATPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracLTo = ((dvalues.fracL[To]*dvalues.distPop[To]) 
                          + graph.nodes[NODE]['HISPLATPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            countB = 0
            countL = 0
            countW = 0

            if newFracWFrom < 0.5:
                if newFracBFrom > max(newFracWFrom,newFracLFrom):
                    countB += 1
                elif newFracLFrom > max(newFracWFrom,newFracBFrom):
                    countL += 1
                else:
                    countW += 1

            if newFracWTo < 0.5:
                if newFracBTo > max(newFracWTo,newFracLTo):
                    countB += 1
                elif newFracLTo > max(newFracWTo,newFracBTo):
                    countL += 1
                else:
                    countW += 1


            numW = 0
            numB = 0
            numL = 0

            if dvalues.fracW[From] < 0.5:
                if dvalues.fracB[From] > max(dvalues.fracW[From],dvalues.fracL[From]):
                    numB += 1
                elif dvalues.fracL[From] > max(dvalues.fracW[From],dvalues.fracB[From]):
                    numL += 1
                else:
                    numW += 1

            if dvalues.fracW[To] < 0.5:
                if dvalues.fracB[To] > max(dvalues.fracW[To],dvalues.fracL[To]):
                    numB += 1
                elif dvalues.fracL[To] > max(dvalues.fracW[To],dvalues.fracB[To]):
                    numL += 1
                else:
                    numW += 1


            if countB != numB or countL != numL or countW != numW:
                checks = False
                updatedValues['why'] = 'demo'
            else:
                updatedValues['fracW'] = [newFracWFrom,newFracWTo]
                updatedValues['fracB'] = [newFracBFrom,newFracBTo]
                updatedValues['fracL'] = [newFracLFrom,newFracLTo]

                
    # If previous checks pass, check demo_basic constraint, if applicable
    if checks:
        if 'demo_basic' in param.constraints:

            # Calculate new % white, Black, Lat/Hisp
            newFracWFrom = ((dvalues.fracW[From]*dvalues.distPop[From]) 
                            - graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[From]-graph.nodes[NODE]['POP20'])
            newFracWTo = ((dvalues.fracW[To]*dvalues.distPop[To])
                          + graph.nodes[NODE]['NHWHITEPOP'])/(dvalues.distPop[To]+graph.nodes[NODE]['POP20'])

            countW = 0

            if newFracWFrom < 0.5:
                countW += 1

            if newFracWTo < 0.5:
                countW += 1


            numW = 0

            if dvalues.fracW[From] < 0.5:
                numW += 1

            if dvalues.fracW[distTo] < 0.5:
                numW += 1


            if countW != numW:
                checks = False
                updatedValues['why'] = 'demo'
            else:
                updatedValues['fracW'] = [newFracWFrom,newFracWTo]

                
    # If previous checks pass, check whole constraint, if applicable, 
    # and only if To is already wholly contained within 1 county
    if checks:
        if ('whole' in param.constraints) and (len(dvalues.distCounties[To]) == 1):

            if graph.nodes[NODE]['GEOID20'][0:5] not in dvalues.distCounties[To]:
                checks = False
                updatedValues['why'] = 'whole'
                    
                    
    # If previous checks pass, check contiguity -- mandatory!
    if checks:
        checks = checkContig(NODE,both,both_aug,graph)
        if not checks:
            updatedValues['why'] = 'contig'
        
        
    # Update votes
    if checks:
        
        newNumDemFrom = dvalues.numDem[From] - graph.nodes[NODE]['VOTES_DEM']
        newNumRepFrom = dvalues.numRep[From] - graph.nodes[NODE]['VOTES_REP']
        newNumDemTo = dvalues.numDem[To] + graph.nodes[NODE]['VOTES_DEM']
        newNumRepTo = dvalues.numRep[To] + graph.nodes[NODE]['VOTES_REP']

        newTVFrom = newNumDemFrom + newNumRepFrom
        newTVTo = newNumDemTo + newNumRepTo

        newFDFrom = newNumDemFrom/newTVFrom
        newFDTo = newNumDemTo/newTVTo

        updatedValues['ND'] = [newNumDemFrom,newNumDemTo]
        updatedValues['NR'] = [newNumRepFrom,newNumRepTo]
        
        
    # If previous checks pass, check cmpttv objective/constraint, if applicable
    if checks:
        if 'cmpttv' in param.constraints:

            # Calculate margins of victory in both districts
            if newTVFrom != 0.0 and newTVTo != 0.0:
                newCmpttvFrom = abs(newNumDemFrom-newNumRepFrom)/newTVFrom
                newCmpttvTo = abs(newNumDemTo-newNumRepTo)/newTVTo
            else:
                newCmpttvFrom = 1.5
                newCmpttvTo = 1.5
                

            # Check whether same number of competitive districts
            if dvalues.compFrac[From] <= param.cmpttv_margin and dvalues.compFrac[To] <= param.cmpttv_margin:
                if newCmpttvFrom > param.cmpttv_margin or newCmpttvTo > param.cmpttv_margin:
                    checks = False
                    updatedValues['why'] = 'cmpttv'

            elif dvalues.compFrac[From] <= param.cmpttv_margin or dvalues.compFrac[To] <= param.cmpttv_margin:
                if newCmpttvFrom > param.cmpttv_margin and newCmpttvTo > param.cmpttv_margin:
                    checks = False
                    updatedValues['why'] = 'cmpttv'
                elif newCmpttvFrom <= param.cmpttv_margin and newCmpttvTo <= param.cmpttv_margin:
                    checks = False
                    updatedValues['why'] = 'cmpttv'

            elif dvalues.compFrac[From] > param.cmpttv_margin and dvalues.compFrac[To] > param.cmpttv_margin:
                if newCmpttvFrom <= param.cmpttv_margin or newCmpttvTo <= param.cmpttv_margin:
                    checks = False
                    updatedValues['why'] = 'cmpttv'

            
                    
    # If previous checks pass, check mm objective/constraint, if applicable
    if checks:
        if 'mm' in param.constraints:

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo

                mm = abs((np.median(newFD[1:])) - (sum(newFD[1:])/((dvalues.numDistricts)-1)))

            else:
                mm = 1.5


            if mm > param.mm_threshold:
                checks = False
                updatedValues['why'] = 'mm'
                

        
    # If previous checks pass, check eg objective/constraint, if applicable
    if checks:
        if 'eg' in param.constraints:

            # (From,To,FDFrom,FDTo,TVFrom,TVTo,num,waste,total)
            if newTVFrom != 0.0 and newTVTo != 0.0:
                eg,wFrom,wTo = update_EG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedNominal,sum(dvalues.totalVotesByDistrict))
            else:
                eg = 1.5


            if eg > param.eg_threshold:
                checks = False
                updatedValues['why'] = 'eg'
            else:
                updatedValues['eg'] = [eg,wFrom,wTo]

    
    # If previous checks pass, check eg_shift objective/constraint, if applicable
    if checks:
        if 'eg_shift' in param.constraints:

            if newTVFrom != 0.0 and newTVTo != 0.0:
                seg,wFrom,wTo = update_ShiftedEG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedShift,sum(dvalues.totalVotesByDistrict))
            else:
                seg = 1.5


            if seg > param.seg_threshold:
                checks = False
                updatedValues['why'] = 'eg_shift'
            else:
                updatedValues['eg_shift'] = [seg,wFrom,wTo]
                
                
    # If previous checks pass, check pa objective/constraint, if applicable
    if checks:
        if 'pa' in param.constraints:

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo
                pa = update_PA(newFD[1:])
            else:
                pa = 1.5


            if pa > param.pa_threshold:
                checks = False
                updatedValues['why'] = 'pa'
                
                
    # If previous checks pass, check for hole -- mandatory!
    if checks:
        checks = checkHole_greedy(NODE,From,To,graph,dvalues)
        if not checks:
            updatedValues['why'] = 'hole'

        
    # Return values
    if checks:
        updatedValues['accept'] = True
    else:
        updatedValues['accept'] = False
        
    return updatedValues



# GREEDY CHECK - objective
# Checks that moving node n from district From to district To improves objective
# Input: district From, district To, chosen node, PARAMS object,
#        PLAN_VALUES object, Gerrychain graph object, current objective value
# Output: updated values dict
def FlipCheck_greedy_obj(From,To,NODE,param,dvalues,graph,obj_current):
    
    # Use bool to keep track of whether objective/constraints are satisfied
    checks = True
    
    # Record updates in case move is accepted
    updatedValues = {}
    
    # Determine intersection of N(chosen node)/N_aug(chosen node) and V(From)
    both = [n for n in graph.nodes[NODE]['neighbors'] if graph.nodes[n]['assignment'] == From]
    both_aug = [n for n in graph.nodes[NODE]['neighbors_aug'] if graph.nodes[n]['assignment'] == From]
    
    # Reject move if node is the only unit in its district
    if len(both) == 0:
        checks = False
        updatedValues['why'] = 'empty'

    # Check pop objective/constraint, if applicable
    if checks:
        if param.objective == 'pop':

            # Determine new district populations
            newPopFrom = dvalues.distPop[From] - graph.nodes[NODE]['POP20']
            newPopTo = dvalues.distPop[To] + graph.nodes[NODE]['POP20']

            percentFrom = abs(1-(newPopFrom/dvalues.meanDistPop))
            percentTo = abs(1-(newPopTo/dvalues.meanDistPop))

            dev = max(percentFrom,percentTo)

            if dev > max(dvalues.distPopPercent[From],dvalues.distPopPercent[To]):
                checks = False
                updatedValues['why'] = 'pop'

            else:
                updatedValues['pop'] = [newPopFrom,newPopTo]
                updatedValues['improvement'] = abs(max(dev,obj_current)-obj_current)


    # If previous checks pass, check perim objective/constraint, if applicable
    if checks:
        if param.objective == 'perim':

            perimFrom = 0
            perimTo = 0
            for nb in graph.nodes[NODE]['neighbors']:
                if graph.nodes[nb]['assignment'] == From:
                    perimFrom += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimFrom += adj[(NODE,nb)]
                elif graph.nodes[nb]['assignment'] == To:
                    perimTo += float(graph.edges[(NODE,nb)]['shared_perim'])/1000
                    #perimTo += adj[(NODE,nb)]

            newPerimFrom = dvalues.distPerimeter[From] + (2*perimFrom - graph.nodes[NODE]['perimeter'])
            newPerimTo = dvalues.distPerimeter[To] + (graph.nodes[NODE]['perimeter'] - 2*perimTo)

            newPerimeter = newPerimFrom + newPerimTo

            if newPerimeter > (dvalues.distPerimeter[From]+dvalues.distPerimeter[To]):
                checks = False
                updatedValues['why'] = 'perim'

            else:
                updatedValues['perim'] = [newPerimFrom,newPerimTo]
                updatedValues['improvement'] = abs((newPerimeter + sum(dvalues.distPerimeter) 
                                                    - dvalues.distPerimeter[From] - dvalues.distPerimeter[To])
                                                   -obj_current)

            
    # If previous checks pass, check cut edges objective/constraint, if applicable
    if checks:
        if param.objective == 'cut_edges':

            cut_orig = []
            cut_new = []
            for nb in graph.nodes[NODE]['neighbors']:
                for u in graph.nodes[nb]['neighbors']:
                    if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_orig:
                        cut_orig.append((nb,u))
                        cut_orig.append((u,nb))
                    if nb == NODE:
                        if To != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if u == NODE:
                        if To != graph.nodes[nb]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))
                    if (nb != NODE) and (u != NODE):
                        if graph.nodes[nb]['assignment'] != graph.nodes[u]['assignment'] and (nb,u) not in cut_new:
                            cut_new.append((nb,u))
                            cut_new.append((u,nb))

            cut_orig_val = len(cut_orig)/2
            cut_new_val = len(cut_new)/2

            newCutEdges = dvalues.CutEdges - cut_orig_val + cut_new_val
            
            if newCutEdges > dvalues.CutEdges:
                checks = False
                updatedValues['why'] = 'cut_edges'

            else:
                updatedValues['cut_edges'] = newCutEdges
                updatedValues['improvement'] = abs(newCutEdges-obj_current)


        
    # Update votes
    if checks:
        
        newNumDemFrom = dvalues.numDem[From] - graph.nodes[NODE]['VOTES_DEM']
        newNumRepFrom = dvalues.numRep[From] - graph.nodes[NODE]['VOTES_REP']
        newNumDemTo = dvalues.numDem[To] + graph.nodes[NODE]['VOTES_DEM']
        newNumRepTo = dvalues.numRep[To] + graph.nodes[NODE]['VOTES_REP']

        newTVFrom = newNumDemFrom + newNumRepFrom
        newTVTo = newNumDemTo + newNumRepTo

        newFDFrom = newNumDemFrom/newTVFrom
        newFDTo = newNumDemTo/newTVTo

        updatedValues['ND'] = [newNumDemFrom,newNumDemTo]
        updatedValues['NR'] = [newNumRepFrom,newNumRepTo]
        
        
    # If previous checks pass, check cmpttv objective/constraint, if applicable
    if checks:
        if param.objective == 'cmpttv':

            # Calculate margins of victory in both districts
            if newTVFrom != 0.0 and newTVTo != 0.0:
                newCmpttvFrom = abs(newNumDemFrom-newNumRepFrom)/newTVFrom
                newCmpttvTo = abs(newNumDemTo-newNumRepTo)/newTVTo
            else:
                newCmpttvFrom = 1.5
                newCmpttvTo = 1.5
                

                
            newNumCmpttv = (obj_current - (dvalues.compFrac[From] <= param.cmpttv_margin) 
                            - (dvalues.compFrac[To] <= param.cmpttv_margin) 
                            + (newCmpttvFrom <= param.cmpttv_margin)
                            + (newCmpttvTo <= param.cmpttv_margin))

            # Check that number of competitive districts did not decrease
            if newNumCmpttv < obj_current:
                checks = False
                updatedValues['why'] = 'cmpttv'

            else:
                updatedValues['improvement'] = abs(newNumCmpttv-obj_current)
                

                    
    # If previous checks pass, check mm objective/constraint, if applicable
    if checks:
        if param.objective == 'mm':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo

                mm = abs((np.median(newFD[1:])) - (sum(newFD[1:])/((dvalues.numDistricts)-1)))

            else:
                mm = 1.5


            if mm > obj_current:
                checks = False
                updatedValues['why'] = 'mm'

            else:
                updatedValues['mm'] = mm
                updatedValues['improvement'] = abs(mm-obj_current)
                

        
    # If previous checks pass, check eg objective/constraint, if applicable
    if checks:
        if param.objective == 'eg':

            # (From,To,FDFrom,FDTo,TVFrom,TVTo,num,waste,total)
            if newTVFrom != 0.0 and newTVTo != 0.0:
                eg,wFrom,wTo = update_EG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedNominal,sum(dvalues.totalVotesByDistrict))
            else:
                eg = 1.5


            if eg > obj_current:
                checks = False
                updatedValues['why'] = 'eg'

            else:
                updatedValues['eg'] = [eg,wFrom,wTo]
                updatedValues['improvement'] = abs(eg-obj_current)

    
    # If previous checks pass, check eg_shift objective/constraint, if applicable
    if checks:
        if param.objective == 'eg_shift':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                seg,wFrom,wTo = update_ShiftedEG_fast_it(From,To,newFDFrom,newFDTo,newTVFrom,newTVTo,dvalues.numDistricts,
                                                 dvalues.wastedShift,sum(dvalues.totalVotesByDistrict))
            else:
                seg = 1.5


            if seg > obj_current:
                checks = False
                updatedValues['why'] = 'eg_shift'

            else:
                updatedValues['eg_shift'] = [seg,wFrom,wTo]
                updatedValues['improvement'] = abs(seg-obj_current)
                
                
    # If previous checks pass, check pa objective/constraint, if applicable
    if checks:
        if param.objective == 'pa':

            if newTVFrom != 0.0 and newTVTo != 0.0:
                newFD = [fd for fd in dvalues.fracDem]
                newFD[From] = newFDFrom
                newFD[To] = newFDTo
                pa = update_PA(newFD[1:])
            else:
                pa = 1.5


            if pa > obj_current:
                checks = False
                updatedValues['why'] = 'pa'

            else:
                updatedValues['pa'] = pa
                updatedValues['improvement'] = abs(pa-obj_current)
                

        
    # Return values
    if checks:
        updatedValues['accept'] = True
    else:
        updatedValues['accept'] = False
        
    return updatedValues



# Input: PARAMS object, PLAN_VALUES object, Gerrychain graph, objective value list, reject reasons list
# Output: Gerrychain graph, objective value list, # iterations, run time, reject count dict
def perform_iterations(param,dvalues,graph,objective_values,reasons):
    
    start = time.time()
    start_cpu = time.process_time()

    finished = False
    k = 0
    numIt = 0
    
    if param.search_type == 'SA':
        SA_run = True
        best_obj = objective_values[-1]
        best_plan = {node:graph.nodes[node]['assignment'] for node in graph.nodes}
        param.temperature = param.temperature_initial
    else:
        SA_run = False

    countRejects = {}
    for re in reasons:
        countRejects[re] = 0

    while not finished:

        # Give user idea of algorithm progress
        k += 1
        #print(k)

        # Check convergence, or complete a pre-determined number of iterations
        if param.converge and k > 1:
            if (abs(objective_values[-2] - objective_values[-1]) < param.convergence_threshold):
                numIt += 1
            else:
                numIt = 0

            if numIt >= param.num_iterations:
                finished = True
                continue

        elif (not param.converge) and (k >= param.num_iterations):
            finished = True


        # Determine unit in consideration for move and its district
        index = int(len(dvalues.borderNodes)*random.random())
        chosen_node = dvalues.borderNodes[index]
        distFrom = graph.nodes[chosen_node]['assignment']

        # Choose district that chosen unit can move too
        candidates = []

        for nb in graph.nodes[chosen_node]['neighbors']:
            otherDist = graph.nodes[nb]['assignment']
            if nb != 'dummy' and otherDist != distFrom and otherDist not in candidates:
                candidates.append(otherDist)

        index = int(len(candidates)*random.random())
        distTo = candidates[index]

#                 print('\nNode: ',chosen_node)
#                 print('distFrom: ',distFrom)
#                 print('distTo: ',distTo)


        # Call flip step check
        if param.search_type == 'SA':
            
            newValues = FlipCheck_SA(distFrom,distTo,chosen_node,param,dvalues,graph,objective_values[-1])
                    
        else:
            newValues = FlipCheck_simple(distFrom,distTo,chosen_node,param,dvalues,graph,objective_values[-1])
                
                

        # Once move is accepted/rejected, update everything
        if newValues['accept']:

            #print('Accepted!\n')

            # Update full record
            graph.nodes[chosen_node]['assignment'] = distTo 

            # Update population, if applicable
            if 'pop' in param.constraints or param.objective == 'pop':
                
                # Update district populations
                dvalues.distPop[distFrom] = newValues['pop'][0]
                dvalues.distPop[distTo] = newValues['pop'][1]

                # Update district percent difference from expected population
                dvalues.distPopPercent[distFrom] = abs(1-(dvalues.distPop[distFrom])/(dvalues.meanDistPop))
                dvalues.distPopPercent[distTo] = abs(1-(dvalues.distPop[distTo])/(dvalues.meanDistPop))

                # Record pop objective value
                if param.objective == 'pop':
                    objective_values.append(max(dvalues.distPopPercent))

            # Update district perimeters, if applicable
            if 'perim' in param.constraints or param.objective == 'perim':
                dvalues.distPerimeter[distFrom] = newValues['perim'][0]
                dvalues.distPerimeter[distTo] = newValues['perim'][1]

                # Record perim objective value
                if param.objective == 'perim':
                    objective_values.append(sum(dvalues.distPerimeter))
                    
            # Update number of cut edges, if applicable
            if 'cut_edges' in param.constraints or param.objective == 'cut_edges':
                dvalues.CutEdges = newValues['cut_edges']
                
                # Record cut edges objective value
                if param.objective == 'cut_edges':
                    objective_values.append(dvalues.CutEdges)

            # Update borderNodes
            toAppend = []
            for vert in graph.nodes[chosen_node]['neighbors']:
                isBorderNode = 0
                distVert = graph.nodes[vert]['assignment']
                for nb in graph.nodes[vert]['neighbors']:
                    otherDist = graph.nodes[nb]['assignment']
                    if otherDist != distVert and otherDist != 0 and distVert != 0:
                        if dvalues.borderNodesYesNo[vert] == 0:
                            toAppend.append(vert)
                        isBorderNode = 1
                        break

                dvalues.borderNodesYesNo[vert] = isBorderNode


            newBN = [bn for bn in dvalues.borderNodes if dvalues.borderNodesYesNo[bn] == 1]
            for bn in toAppend:
                newBN.append(bn)
                
            dvalues.borderNodes = [bn for bn in newBN]
            newBN = []

            # Update votes
            dvalues.numDem[distFrom] = newValues['ND'][0]
            dvalues.numDem[distTo] = newValues['ND'][1]

            dvalues.numRep[distFrom] = newValues['NR'][0]
            dvalues.numRep[distTo] = newValues['NR'][1]

            dvalues.totalVotesByDistrict[distFrom] = dvalues.numDem[distFrom]+dvalues.numRep[distFrom]
            dvalues.totalVotesByDistrict[distTo] = dvalues.numDem[distTo]+dvalues.numRep[distTo]

            dvalues.fracDem[distFrom] = dvalues.numDem[distFrom]/dvalues.totalVotesByDistrict[distFrom]
            dvalues.fracDem[distTo] = dvalues.numDem[distTo]/dvalues.totalVotesByDistrict[distTo]

            # Update % white, Black, Lat/Hisp
            if 'demo' in param.constraints:
                dvalues.fracW[distFrom] = newValues['fracW'][0]
                dvalues.fracB[distFrom] = newValues['fracB'][0]
                dvalues.fracL[distFrom] = newValues['fracL'][0]

                dvalues.fracW[distTo] = newValues['fracW'][1]
                dvalues.fracB[distTo] = newValues['fracB'][1]
                dvalues.fracL[distTo] = newValues['fracL'][1]
                
            if 'demo_basic' in param.constraints:
                dvalues.fracW[distFrom] = newValues['fracW'][0]
                dvalues.fracW[distTo] = newValues['fracW'][1]

            # Update cmpttv, if applicable
            if 'cmpttv' in param.constraints or param.objective == 'cmpttv':

                dvalues.compFrac[distFrom] = abs(2*dvalues.fracDem[distFrom] - 1)
                dvalues.compFrac[distTo] = abs(2*dvalues.fracDem[distTo] - 1)

                # Record cmpttv objective value
                if param.objective == 'cmpttv':
                    objective_values.append(len([f for f in dvalues.compFrac[1:] if f <= param.cmpttv_margin]))

            # Update MM, if applicable
            if param.objective == 'mm':
                objective_values.append(newValues['mm'])
                #print('newValues[mm] = ',newValues['mm'])

            # Update EG, if applicable
            if 'eg' in param.constraints or param.objective == 'eg':

                dvalues.wastedNominal[distFrom] = newValues['eg'][1]
                dvalues.wastedNominal[distTo] = newValues['eg'][2]

                # Record eg objective value
                if param.objective == 'eg':
                    objective_values.append(newValues['eg'][0])

            # Updated Shifted EG, if applicable
            if 'eg_shift' in param.constraints or param.objective == 'eg_shift':

                for s in [-0.05,-0.04,-0.03,-0.02,-0.01,0.0,0.01,0.02,0.03,0.04,0.05]:
                    dvalues.wastedShift[(distFrom,s)] = newValues['eg_shift'][1][s]
                    dvalues.wastedShift[(distTo,s)] = newValues['eg_shift'][2][s]

                # Record eg_shift objective value
                if param.objective == 'eg_shift':
                    objective_values.append(newValues['eg_shift'][0])

            # Update PA, if applicable
            if param.objective == 'pa':
                objective_values.append(newValues['pa'])

                
            # Update SA temperature, if applicable
            if param.search_type == 'SA':
                
                if newValues['worsen']:
                    param.temperature = param.temperature * param.cooling
                    #print('Temp: ',param.temperature)
                    if param.temperature < param.epsilon:
                        param.search_type = 'simple'
            
            # Update best plan and best obj, if applicable
            if SA_run:
                
                if param.objective == 'cmpttv' and objective_values[-1] >= best_obj:
                    best_obj = objective_values[-1]
                    best_plan = {node:graph.nodes[node]['assignment'] for node in graph.nodes}
                elif param.objective != 'cmpttv' and objective_values[-1] <= best_obj:
                    best_obj = objective_values[-1]
                    best_plan = {node:graph.nodes[node]['assignment'] for node in graph.nodes}

        else:
            #print('Rejected! because ',newValues['why'])
#                     if newValues['why'] == 'hole':
#                         print('hole')

            countRejects[newValues['why']] += 1
            temp = objective_values[-1]
            objective_values.append(temp)
            
            
        # Stop running if exceeding time limit
        if param.time_limit_bool and (time.time()-param.time_start > param.time_limit):
            finished = True

   
    # If SA, make final plan the plan with the best recorded objective value
    if SA_run:
        param.search_type = 'SA'
        objective_values.append(best_obj)
        for node in graph.nodes:
            graph.nodes[node]['assignment'] = best_plan[node]

    return graph, objective_values, k, time.time()-start, time.process_time()-start_cpu, countRejects



# Input: PARAMS object, PLAN_VALUES object, Gerrychain graph, objective value list, reject reasons list
# Output: Gerrychain graph, objective value list, # iterations, run time, reject count dict
def perform_iterations_greedy(param,dvalues,graph,objective_values,reasons):
    
    start = time.time()
    start_cpu = time.process_time()

    finished = False
    k = 0
    numIt = 0

    countRejects = {}
    for re in reasons:
        countRejects[re] = 0

    while not finished:

        # Give user idea of algorithm progress
        k += 1
        
        if k % 100 == 0:
            print(k)

        # Check convergence, or complete a pre-determined number of iterations
        if param.converge and k > 1:
            if (abs(objective_values[-2] - objective_values[-1]) < param.convergence_threshold):
                numIt += 1
            else:
                numIt = 0

            if numIt >= param.num_iterations:
                finished = True
                continue

        elif (not param.converge) and (k >= param.num_iterations):
            finished = True

            
            
        # Check all border nodes to determine best improvement
        improvement_candidates = []
        best_move = ['Nothing','Nothing',0]
        
        # First check for objective improvement
        for chosen_node in dvalues.borderNodes:
            
            distFrom = graph.nodes[chosen_node]['assignment']
            
            # Find districts that chosen unit can move too
            candidates = []

            for nb in graph.nodes[chosen_node]['neighbors']:
                otherDist = graph.nodes[nb]['assignment']
                if nb != 'dummy' and otherDist != distFrom and otherDist not in candidates:
                    candidates.append(otherDist)
                    
            # Check if node-district pair yields improvement
            for distTo in candidates:
                
                newValues = FlipCheck_greedy_obj(distFrom,distTo,chosen_node,param,dvalues,graph,objective_values[-1])
                
                if newValues['accept']:
                    improvement_candidates.append((chosen_node,distTo,newValues['improvement']))
                
                
                
        # Then check for feasibility
        for chosen_node,distTo,improvement in improvement_candidates:
            
            distFrom = graph.nodes[chosen_node]['assignment']
            
            newValues = FlipCheck_greedy_feasible(distFrom,distTo,chosen_node,param,dvalues,graph)
                
            if newValues['accept']:

                if improvement > best_move[2]:

                    best_move = [chosen_node,distTo,improvement]

                elif improvement == best_move[2]:

                    if best_move[0] == 'Nothing':
                        best_move = [chosen_node,distTo,improvement]
                    elif random.random() < 0.5:
                        best_move = [chosen_node,distTo,improvement]

            
            

        # Set best move
        if best_move[0] == 'Nothing':
            finished = True
            break
        
        chosen_node = best_move[0]
        distFrom = graph.nodes[chosen_node]['assignment']
        distTo = best_move[1]
        
    
    
            
#         # Determine unit in consideration for move and its district
#         index = int(len(dvalues.borderNodes)*random.random())
#         chosen_node = dvalues.borderNodes[index]
#         distFrom = graph.nodes[chosen_node]['assignment']

#         # Choose district that chosen unit can move too
#         candidates = []

#         for nb in graph.nodes[chosen_node]['neighbors']:
#             otherDist = graph.nodes[nb]['assignment']
#             if nb != 'dummy' and otherDist != distFrom and otherDist not in candidates:
#                 candidates.append(otherDist)

#         index = int(len(candidates)*random.random())
#         distTo = candidates[index]

#                 print('\nNode: ',chosen_node)
#                 print('distFrom: ',distFrom)
#                 print('distTo: ',distTo)


        # Call flip step check
        newValues = FlipCheck_simple(distFrom,distTo,chosen_node,param,dvalues,graph,objective_values[-1])
                
                

        # Once move is accepted/rejected, update everything
        if newValues['accept']:

            #print('Accepted!\n')

            # Update full record
            graph.nodes[chosen_node]['assignment'] = distTo 

            # Update population, if applicable
            if 'pop' in param.constraints or param.objective == 'pop':
                
                # Update district populations
                dvalues.distPop[distFrom] = newValues['pop'][0]
                dvalues.distPop[distTo] = newValues['pop'][1]

                # Update district percent difference from expected population
                dvalues.distPopPercent[distFrom] = abs(1-(dvalues.distPop[distFrom])/(dvalues.meanDistPop))
                dvalues.distPopPercent[distTo] = abs(1-(dvalues.distPop[distTo])/(dvalues.meanDistPop))

                # Record pop objective value
                if param.objective == 'pop':
                    objective_values.append(max(dvalues.distPopPercent))

            # Update district perimeters, if applicable
            if 'perim' in param.constraints or param.objective == 'perim':
                dvalues.distPerimeter[distFrom] = newValues['perim'][0]
                dvalues.distPerimeter[distTo] = newValues['perim'][1]

                # Record perim objective value
                if param.objective == 'perim':
                    objective_values.append(sum(dvalues.distPerimeter))
                    
            # Update number of cut edges, if applicable
            if 'cut_edges' in param.constraints or param.objective == 'cut_edges':
                dvalues.CutEdges = newValues['cut_edges']
                
                # Record cut edges objective value
                if param.objective == 'cut_edges':
                    objective_values.append(dvalues.CutEdges)

            # Update borderNodes
            toAppend = []
            for vert in graph.nodes[chosen_node]['neighbors']:
                isBorderNode = 0
                distVert = graph.nodes[vert]['assignment']
                for nb in graph.nodes[vert]['neighbors']:
                    otherDist = graph.nodes[nb]['assignment']
                    if otherDist != distVert and otherDist != 0 and distVert != 0:
                        if dvalues.borderNodesYesNo[vert] == 0:
                            toAppend.append(vert)
                        isBorderNode = 1
                        break

                dvalues.borderNodesYesNo[vert] = isBorderNode


            newBN = [bn for bn in dvalues.borderNodes if dvalues.borderNodesYesNo[bn] == 1]
            for bn in toAppend:
                newBN.append(bn)
                
            dvalues.borderNodes = [bn for bn in newBN]
            newBN = []

            # Update votes
            dvalues.numDem[distFrom] = newValues['ND'][0]
            dvalues.numDem[distTo] = newValues['ND'][1]

            dvalues.numRep[distFrom] = newValues['NR'][0]
            dvalues.numRep[distTo] = newValues['NR'][1]

            dvalues.totalVotesByDistrict[distFrom] = dvalues.numDem[distFrom]+dvalues.numRep[distFrom]
            dvalues.totalVotesByDistrict[distTo] = dvalues.numDem[distTo]+dvalues.numRep[distTo]

            dvalues.fracDem[distFrom] = dvalues.numDem[distFrom]/dvalues.totalVotesByDistrict[distFrom]
            dvalues.fracDem[distTo] = dvalues.numDem[distTo]/dvalues.totalVotesByDistrict[distTo]

            # Update % white, Black, Lat/Hisp
            if 'demo' in param.constraints:
                dvalues.fracW[distFrom] = newValues['fracW'][0]
                dvalues.fracB[distFrom] = newValues['fracB'][0]
                dvalues.fracL[distFrom] = newValues['fracL'][0]

                dvalues.fracW[distTo] = newValues['fracW'][1]
                dvalues.fracB[distTo] = newValues['fracB'][1]
                dvalues.fracL[distTo] = newValues['fracL'][1]
                
            if 'demo_basic' in param.constraints:
                dvalues.fracW[distFrom] = newValues['fracW'][0]
                dvalues.fracW[distTo] = newValues['fracW'][1]

            # Update cmpttv, if applicable
            if 'cmpttv' in param.constraints or param.objective == 'cmpttv':

                dvalues.compFrac[distFrom] = abs(2*dvalues.fracDem[distFrom] - 1)
                dvalues.compFrac[distTo] = abs(2*dvalues.fracDem[distTo] - 1)

                # Record cmpttv objective value
                if param.objective == 'cmpttv':
                    objective_values.append(len([f for f in dvalues.compFrac[1:] if f <= param.cmpttv_margin]))

            # Update MM, if applicable
            if param.objective == 'mm':
                objective_values.append(newValues['mm'])
                #print('newValues[mm] = ',newValues['mm'])

            # Update EG, if applicable
            if 'eg' in param.constraints or param.objective == 'eg':

                dvalues.wastedNominal[distFrom] = newValues['eg'][1]
                dvalues.wastedNominal[distTo] = newValues['eg'][2]

                # Record eg objective value
                if param.objective == 'eg':
                    objective_values.append(newValues['eg'][0])

            # Updated Shifted EG, if applicable
            if 'eg_shift' in param.constraints or param.objective == 'eg_shift':

                for s in [-0.05,-0.04,-0.03,-0.02,-0.01,0.0,0.01,0.02,0.03,0.04,0.05]:
                    dvalues.wastedShift[(distFrom,s)] = newValues['eg_shift'][1][s]
                    dvalues.wastedShift[(distTo,s)] = newValues['eg_shift'][2][s]

                # Record eg_shift objective value
                if param.objective == 'eg_shift':
                    objective_values.append(newValues['eg_shift'][0])

            # Update PA, if applicable
            if param.objective == 'pa':
                objective_values.append(newValues['pa'])

            

        else:
            #print('Rejected! because ',newValues['why'])
#                     if newValues['why'] == 'hole':
#                         print('hole')

            countRejects[newValues['why']] += 1
            temp = objective_values[-1]
            objective_values.append(temp)
            
            
        # Stop running if exceeding time limit
        if param.time_limit_bool and (time.time()-param.time_start > param.time_limit):
            finished = True

   

    return graph, objective_values, k, time.time()-start, time.process_time()-start_cpu, countRejects



# Input: PARAMS object
# Output: none
def FlipLocalSearch(param):
    
    # Load state data
    print('\nLoading state data now (shapefile may take a minute or two to load, depending on size)')
    print('...')
    graph,adj,cutV = make_graph(param)
    print('\nDone!\n')

    # Output file for why flip proposals got rejected
    outRejects = open(param.OutputFolder+'/Flip_'+param.StateFolder
                      +'_'+param.objective+'Obj_'+str(SEED)+'Seed_Rejections.csv','w')
    writerRejects = csv.writer(outRejects,delimiter=',')
    reasons = ['runtime','cpu time','empty','contig','hole','pop','demo_basic','demo','whole',
               'perim','cut_edges','eg','eg_shift','mm','pa','cmpttv']
    writerRejects.writerow(['Final Plan']+reasons)
    
    # Record start time
    param.time_start = time.time()
    
    
    # Iterate through plans
    for i in range(0,param.repeated_runs):
    
        for file in param.maps:
            
            print('\nGiven district map: ',file)
            
            # Create PLAN_VALUES object to store district info
            dvalues = PLAN_VALUES()

            # Read in district plan
            planFile = open(param.PlanFolder + '/' + file,'r')
            reader = csv.reader(planFile,delimiter=',')

            labels = next(reader)
            plan = {}
            count = 0
            for line in reader:
                if line != [] and line != ['','']:
                    plan[line[0]] = int(line[1])
                    if int(line[1]) > count:
                        count = int(line[1])

            planFile.close()
            #print(plan)
            
            dvalues.numDistricts = count + 1
            
            # Make assignment attribute for nodes in graph object
            for node in graph.nodes:
                if node == 'dummy':
                    graph.nodes[node]['assignment'] = 0
                else:
                    graph.nodes[node]['assignment'] = plan[graph.nodes[node]['GEOID20']]
            
            
            # Get district info
            obj_list,dvalues = get_district_info(param,dvalues,graph,adj)
            
            print('\nRunning iterations now\n...')
            
            # Run iterations
            if param.search_type == 'greedy':
                final_graph,final_obj,final_it,runtime,time_cpu,final_count_rejects = perform_iterations_greedy(param,dvalues,graph,obj_list,reasons)
            else:
                final_graph,final_obj,final_it,runtime,time_cpu,final_count_rejects = perform_iterations(param,dvalues,graph,obj_list,reasons)
            

            # Plot objective values
            plt.plot(final_obj)
            plt.show()
            print('Final ',param.objective,' = ',final_obj[-1])

            # Print run time
            print('Runtime = ',np.round(runtime,3),' seconds')
            print('CPU time = ',np.round(time_cpu,3),' seconds')
            
            # Output optimized plan
            constraint_str = ''
            for c in param.constraints:
                constraint_str += c

            outFile_str = param.output_name+'_'+str(SEED)+'Seed_'+str(final_it)+'It_'+str(np.round(runtime,3))+'seconds_'+str(param.edgeBonus)+'B_'+str(param.edgePenalty)+'P_'+str(param.objective)+'Obj'+str(np.round(final_obj[-1],5))+'_'+constraint_str+'.csv'
            outFile = open(param.OutputFolder + '/' + outFile_str,'w')

            #final_assignment = {}
            writer = csv.writer(outFile,delimiter=',')

            writer.writerow(['GEOID20','district'])
            for node in graph.nodes:
                if node != 'dummy':
                    #final_assignment[graph.nodes[node]['GEOID20']] = graph.nodes[node]['assignment']
                    writer.writerow([graph.nodes[node]['GEOID20'],graph.nodes[node]['assignment']])
                    if node in cutV:
                        for u in cutV[node]:
                            #final_assignment[u] = graph.nodes[node]['assignment']
                            writer.writerow([u,graph.nodes[node]['assignment']])

            outFile.close()
            
            final_count_rejects['runtime'] = np.round(runtime,3)
            final_count_rejects['cpu time'] = np.round(time_cpu,3)
            writerRejects.writerow([outFile_str]+[final_count_rejects[val] for val in final_count_rejects])
            
            
            # Stop running if exceeding time limit
            if param.time_limit_bool and (time.time()-param.time_start > param.time_limit):
                outRejects.close()
                print('\n\nDone running plans! (reached time limit)')
                return
                

        
    outRejects.close()
    print('\n\nDone running plans!')
    
    return
        
    #return final_assignment

In [ ]:
parameters = read_parameters('Flip_PARAMETERS_2023.csv')
if type(parameters) == type(0):
    print('Whoops!\n\n')
else:
    FlipLocalSearch(parameters)
